# 4.2 SQL Intermediate - Table Relations and joins

1. Joining Data In SQL
    
    Learn how to use joins to query data across two tables
    * What joins are and how to use the four common join types
    * How to use joins with subqueries to write intermediate queries
    * How to use joins to answer business questions
    
2. Intermediate Joins In SQL
    
    Learn how to join data from more complex databases and write more complex queries
    * Understand how to write queries that join three or more tables
    * Learn recursive joins, pattern matching and case statement
    * Practice writing advanced joins
    
3. Building And Organizing Complex Queries
    
    Learn techniques to organize your queries to make them easier to read and understand
    * How to format queries to make them readable
    * How to use the 'with' clause and views to organize your queries
    * How to use set operations to combine tables
    
4. Guided Project: Answering Business Questions Using SQL
    
    Learn and practice the SQL data analysis workflow
    * How to frame a business question into advanced SQL queries
    * Use pandasplot methods to create visualizations of your query results
    * How to write analysis to communicate the results of your work
    
5. Table Relations and Normalization
    
    Learn how to create and modify databases
    * How to use the SQLite shell to work with databases
    * How to create and normalize database tables
    * How to insert and update in database tables
    
6. Guided Project: Designing And Creating A Database
    
    Learn how to design, create, and populate a normalized SQL database
    * How to design a normalized database schema
    * How to create normalized database tables
    * How to import data into a database and transform it into normalized tables
    

## 4.2.1 Joining Data in SQL

<span style="color:red">Primary Key; Foreign Key</span>

### 4.2.1.1 Introducing Joins


In the SQL Fundamentals course, we worked exclusively with data that existed in a __single__ table. In the real world, it's much more common for databases to have data in more than one table. If we want to be able to work with that data, we'll have to combine multiple tables within a query. The way we do this in SQL is using __joins__. As in the SQL Fundamentals course, we'll continue to use __[SQLite](https://sqlite.org/index.html)__ throughout this course.

In this mission, we're going to be using a version of the CIA World Factbook (Factbook) database from the guided project from the SQL Fundamentals course. To refresh your memory, this database had one table called __facts__, where each row represented a country from the Factbook. Here are the first 5 rows of the facts table:

<table class="dataframe">
<thead>
<tr>
<th>id</th>
<th>code</th>
<th>name</th>
<th>area</th>
<th>area_land</th>
<th>area_water</th>
<th>population</th>
<th>population_growth</th>
<th>birth_rate</th>
<th>death_rate</th>
<th>migration_rate</th>
</tr>
</thead>
<tbody>
<tr>
<td>1</td>
<td>af</td>
<td>Afghanistan</td>
<td>652230</td>
<td>652230</td>
<td>0</td>
<td>32564342</td>
<td>2.32</td>
<td>38.57</td>
<td>13.89</td>
<td>1.51</td>
</tr>
<tr>
<td>2</td>
<td>al</td>
<td>Albania</td>
<td>28748</td>
<td>27398</td>
<td>1350</td>
<td>3029278</td>
<td>0.30</td>
<td>12.92</td>
<td>6.58</td>
<td>3.30</td>
</tr>
<tr>
<td>3</td>
<td>ag</td>
<td>Algeria</td>
<td>2381741</td>
<td>2381741</td>
<td>0</td>
<td>39542166</td>
<td>1.84</td>
<td>23.67</td>
<td>4.31</td>
<td>0.92</td>
</tr>
<tr>
<td>4</td>
<td>an</td>
<td>Andorra</td>
<td>468</td>
<td>468</td>
<td>0</td>
<td>85580</td>
<td>0.12</td>
<td>8.13</td>
<td>6.96</td>
<td>0.00</td>
</tr>
<tr>
<td>5</td>
<td>ao</td>
<td>Angola</td>
<td>1246700</td>
<td>1246700</td>
<td>0</td>
<td>19625353</td>
<td>2.78</td>
<td>38.78</td>
<td>11.49</td>
<td>0.46</td>
</tr>
</tbody>
</table>

In addition to the facts table we've added a new table, called __cities__ which contains information on __[major urban areas](https://www.cia.gov/library/publications/the-world-factbook/docs/notesanddefs.html?fieldkey=2219&term=Major%20urban%20areas%20-%20population)__ from countries in the Factbook (for the rest of this mission, we'll use the word 'cities' to mean the same as 'major urban areas'). Let's take a look at the first few rows of this new table and a description of what each column represents:

<table class="dataframe">
<thead>
<tr>
<th>id</th>
<th>name</th>
<th>population</th>
<th>capital</th>
<th>facts_id</th>
</tr>
</thead>
<tbody>
<tr>
<td>1</td>
<td>Oranjestad</td>
<td>37000</td>
<td>1</td>
<td>216</td>
</tr>
<tr>
<td>2</td>
<td>Saint John'S</td>
<td>27000</td>
<td>1</td>
<td>6</td>
</tr>
<tr>
<td>3</td>
<td>Abu Dhabi</td>
<td>942000</td>
<td>1</td>
<td>184</td>
</tr>
<tr>
<td>4</td>
<td>Dubai</td>
<td>1978000</td>
<td>0</td>
<td>184</td>
</tr>
<tr>
<td>5</td>
<td>Sharjah</td>
<td>983000</td>
<td>0</td>
<td>184</td>
</tr>
</tbody>
</table>

* id - A unique ID for each city.
* name - The name of the city.
* population - The population of the city.
* capital - Whether the city is a capital city: 1 if it is, 0 if it isn't.
* facts_id - The ID of the country, from the facts table.

The last column is of particular interest to us, as it is a column of data that also exists in our original facts table. This __link between tables__ is important as it's used to combine the data in our queries. Below is a __schema diagram__, which shows the two tables in our database, the columns within them and how the two are linked.

![img alt](https://s3.amazonaws.com/dq-content/179/schema.svg)


The line in the schema diagram clearly shows the link between the __id__ column in the __facts table__ and the __facts_id__ column in the __cities table__. You may need to refer back to this schema diagram throughout the mission.

The most common way to join data using SQL is using an __inner join__. The syntax for an inner join is:

```SQL
SELECT [column_names] 
FROM [table_name_one]
INNER JOIN [table_name_two] ON [join_constraint];
```

The inner join clause is made up of two parts:

* INNER JOIN, which tells the SQL engine the name of the table you wish to join in your query, and that you wish to use an inner join.
* ON, which tells the SQL engine what columns to use to join the two tables.

Joins are usually used in a query after the __FROM clause__. Let's look at a basic inner join where we combine the data from both of our tables.

```SQL
SELECT * 
FROM facts
INNER JOIN cities ON cities.facts_id = facts.id
LIMIT 5;
```

Let's look at the line of the query with the join in it:

* INNER JOIN cities - This tells the SQL engine that we wish to join the cities table to our query using an inner join.
* ON cities.facts_id = facts.id - This tells the SQL engine which columns to use when joining the data, following the syntax table_name.column_name.

You might presume that SELECT * FROM facts will mean that the query returns only columns from the facts table, however the * wildcard when used with a join will give you __all columns from both tables__. Here is the result of this query:

<table class="dataframe">
<thead>
<tr>
<th>id</th>
<th>code</th>
<th>name</th>
<th>area</th>
<th>area_land</th>
<th>area_water</th>
<th>population</th>
<th>population_growth</th>
<th>birth_rate</th>
<th>death_rate</th>
<th>migration_rate</th>
<th>id</th>
<th>name</th>
<th>population</th>
<th>capital</th>
<th>facts_id</th>
</tr>
</thead>
<tbody>
<tr>
<td>216</td>
<td>aa</td>
<td>Aruba</td>
<td>180</td>
<td>180</td>
<td>0</td>
<td>112162</td>
<td>1.33</td>
<td>12.56</td>
<td>8.18</td>
<td>8.92</td>
<td>1</td>
<td>Oranjestad</td>
<td>37000</td>
<td>1</td>
<td>216</td>
</tr>
<tr>
<td>6</td>
<td>ac</td>
<td>Antigua and Barbuda</td>
<td>442</td>
<td>442</td>
<td>0</td>
<td>92436</td>
<td>1.24</td>
<td>15.85</td>
<td>5.69</td>
<td>2.21</td>
<td>2</td>
<td>Saint John'S</td>
<td>27000</td>
<td>1</td>
<td>6</td>
</tr>
<tr>
<td>184</td>
<td>ae</td>
<td>United Arab Emirates</td>
<td>83600</td>
<td>83600</td>
<td>0</td>
<td>5779760</td>
<td>2.58</td>
<td>15.43</td>
<td>1.97</td>
<td>12.36</td>
<td>3</td>
<td>Abu Dhabi</td>
<td>942000</td>
<td>1</td>
<td>184</td>
</tr>
<tr>
<td>184</td>
<td>ae</td>
<td>United Arab Emirates</td>
<td>83600</td>
<td>83600</td>
<td>0</td>
<td>5779760</td>
<td>2.58</td>
<td>15.43</td>
<td>1.97</td>
<td>12.36</td>
<td>4</td>
<td>Dubai</td>
<td>1978000</td>
<td>0</td>
<td>184</td>
</tr>
<tr>
<td>184</td>
<td>ae</td>
<td>United Arab Emirates</td>
<td>83600</td>
<td>83600</td>
<td>0</td>
<td>5779760</td>
<td>2.58</td>
<td>15.43</td>
<td>1.97</td>
<td>12.36</td>
<td>5</td>
<td>Sharjah</td>
<td>983000</td>
<td>0</td>
<td>184</td>
</tr>
</tbody>
</table>

This query gives us all columns from both tables and every row where there is a match between the id column from facts and the facts_id from cities, limited to the first 5 rows. We'll look at how the join itself works in detail in a moment, but first let's practice writing our first join.

#### Instructions
* Write a query that returns all columns from the facts and cities tables.
    * Use an INNER JOIN to join the cities table to the facts table.
    * Join the tables on the values where facts.id and cities.facts_id are equal.
    * Limit the query to the first 10 rows.


#### Answers
```SQL
SELECT *
FROM facts 
INNER JOIN cities
ON facts.id =cities.facts_id
LIMIT 10;
```


### 4.2.1.2 Understanding Inner Joins

We've now joined the two tables to give us extra information about each row in cities. Let's take a closer look at how this inner join works.

An inner join works by including only rows from each table that have a match as specified using the __ON__ clause. Let's look at a diagram of how our join from the previous screen works. We have included a selection of rows which best illustrate the join:

![img alt](https://s3.amazonaws.com/dq-content/179/inner_join.svg)

Our inner join will include:

* Rows from the cities table that have a cities.facts_id that matches a facts.id from facts.

Our inner join will not include:

* Rows from the cities table that have a cities.facts_id that doesn't match any facts.id from facts.
* Rows from the facts table that have a facts.id that doesn't match any cities.facts_id from cities.

You can see this represented as a Venn diagram:

![img alt](https://s3.amazonaws.com/dq-content/179/venn_inner.svg)

In the SQL fundamentals course, we learned how to use __[aliases](https://www.tutorialspoint.com/sqlite/sqlite_alias_syntax.htm)__ to specify custom names for columns, eg:

```SQL
SELECT AVG(population) AS AVERAGE_POPULATION
```

We can also create aliases for table names, which makes queries with joins easier to both read and write. Instead of:

```SQL
SELECT * FROM facts
INNER JOIN cities ON cities.facts_id = facts.id
```

We can write:

```SQL
SELECT * FROM facts AS f
INNER JOIN cities AS c ON c.facts_id = f.id
```

Just like with column names, using AS is optional. We can get the same result by writing:

```SQL
SELECT * FROM facts f
INNER JOIN cities c ON c.facts_id = f.id
```

We can also __combine aliases with wildcards__ - for instance, using the aliases created above, __c.*__ would give us all columns from the table cities.

While our query from the previous screen included both columns from the ON clause, we don't need to use either column from our ON clause in our final list of columns. This is useful as it means we can __show only the information we're interested in__, rather than having to include the two join columns every time.

Let's use what we've learned to build on our original query.


#### Instructions
* Write a query that:
    * Joins cities to facts using an INNER JOIN.
    * Uses aliases for table names.
    * Includes, in order:
        * All columns from cities.
        * The name column from facts aliased to country_name.
    * Includes only the first 5 rows.

#### Answers
```SQL
SELECT c.*, f.name country_name 
FROM facts f
INNER JOIN cities c 
ON c.facts_id = f.id
LIMIT 5;
```



In [1]:
import sqlite3

conn=sqlite3.connect('factbook.db')
query = '''
SELECT c.*, f.name country_name 
FROM facts f
INNER JOIN cities c 
ON c.facts_id = f.id
LIMIT 5;
'''

conn.execute(query).fetchall()

[(1, 'Oranjestad', 37000, 1, '216', 'Aruba'),
 (2, "Saint John'S", 27000, 1, '6', 'Antigua and Barbuda'),
 (3, 'Abu Dhabi', 942000, 1, '184', 'United Arab Emirates'),
 (4, 'Dubai', 1978000, 0, '184', 'United Arab Emirates'),
 (5, 'Sharjah', 983000, 0, '184', 'United Arab Emirates')]

### 4.2.1.3 Practicing Inner Joins

Let's practice writing a query to answer a question from our database using an inner join. Say we want to produce a table of countries and their capital cities from our database using what we've learned so far. Our first step is to think about what columns we'll need in our final query. We'll need:

* The name column from facts
* The name column from cities

Given that we've identified that we need data from two tables, we need to think about how to join them. The __schema diagram__ from earlier indicated that there is __only one column in each table that links them together__, so we can use an inner join with those columns to join the data.

So far, thinking through our question we can already write most of our query:

```SQL
SELECT f.name, c.name FROM cities c
INNER JOIN facts f ON f.id = c.facts_id
```

The last part of our process is to make sure we have the correct rows. From the previous two screens we know that a query like this will return all rows from cities that have a corresponding match from facts in the facts_id column. We're only interested in the capital cities from the cities table, so we'll need to use a WHERE clause on the capital column, which has a value of 1 if the city is a capital, and 0 if it isn't:

```SQL
WHERE c.capital = 1
```

We can now put this all together to write a query that answers our question.


#### Instructions
* Write a query that returns, in order:
    * A column of country names, called country.
    * A column of each country's capital city, called capital_city.
* Use an INNER JOIN to join the two tables in your query.


#### Answers
```SQL
SELECT f.name country, c.name capital_city 
FROM cities c
INNER JOIN facts f 
ON f.id = c.facts_id
WHERE c.capital = 1
```


In [3]:
query = '''
SELECT f.name country, c.name capital_city 
FROM cities c
INNER JOIN facts f 
ON f.id = c.facts_id
WHERE c.capital = 1
'''

conn.execute(query).fetchmany(10)

[('Aruba', 'Oranjestad'),
 ('Antigua and Barbuda', "Saint John'S"),
 ('United Arab Emirates', 'Abu Dhabi'),
 ('Afghanistan', 'Kabul'),
 ('Algeria', 'Algiers'),
 ('Azerbaijan', 'Baku'),
 ('Albania', 'Tirana'),
 ('Armenia', 'Yerevan'),
 ('Andorra', 'Andorra La Vella'),
 ('Angola', 'Luanda')]

### 4.2.1.4 Left Joins (查找不匹配的数据)

As we mentioned earlier, an inner join will not include any rows where there is not a mutual match from both tables. This means there could be information we are not seeing in our query where rows don't match.

We can use the SQL console to run some queries to explore this:

```SQL
>>> SELECT COUNT(DISTINCT(name)) FROM facts;
​
    [["COUNT(DISTINCT(name))"], [261]]
​
>>> SELECT COUNT(DISTINCT(facts_id)) FROM cities;
​
    [["COUNT(DISTINCT(facts_id))"], [210]]
```

By running these two queries, we can see that there are some countries in the facts table that don't have corresponding cities in the cities table, which indicates we may have some incomplete data.

Let's look at how we can create a query to explore the missing data using a new type of join— the __left join__.

A left join includes all the rows that an inner join will select, plus any joins from the first (or left) table that don't have a match in the second table. We can see this represented as a Venn diagram.

![img alt](https://s3.amazonaws.com/dq-content/179/venn_left.svg)

Let's look at an example by replacing INNER JOIN with LEFT JOIN from the first query we wrote, and looking at the same selection of rows from our earlier diagram

```SQL
SELECT * FROM facts
LEFT JOIN cities ON cities.facts_id = facts.id
```

![img alt](https://s3.amazonaws.com/dq-content/179/left_join.svg)

Here we can see that for the rows where facts.id doesn't match any values in cities.facts_id (237, 238, 240, and 244), the rows are still included in the results. <span style="color:red">__When this happens__, __all of the columns__ from the cities table are populated with __null__ values</span>.

We can use these __null values__ to __filter__ our results to just the countries that don't exist in cities with a WHERE clause. When making a comparison to null in SQL, we use the __IS keyword__, rather than the = sign. If we want to select rows where a column is null we can write:

```SQL
WHERE column_name IS NULL
```

If we want to select rows where a column name isn't null, we use:

```SQL
WHERE column_name IS NOT NULL
```

Let's use a left join to explore the countries that don't exist in the cities table.


#### Instructions
* Write a query that returns the countries that don't exist in cities:
    * Your query should return two columns:
        * The country names, with the alias country.
        * The country population.
    * Use a LEFT JOIN to join cities to facts.
    * Include only the countries from facts that don't have a corresponding value in cities.

#### Answers
```SQL
SELECT f.name country, f.population
FROM facts f
LEFT JOIN cities c ON c.facts_id = f.id
WHERE c.name IS NULL;
```


### 4.2.1.5 Right Joins and Outer Joins

Looking through the results of the query we wrote in the previous screen, we can see a number of different reasons that countries don't have corresponding values in __cities__:

* Countries with small populations and/or no major urban areas (which are defined as having populations of over 750,000), eg San Marino, Kosovo, and Nauru.
* City-states, such as Monaco and Singapore.
* Territories that are not themselves countries, such as Hong Kong, Gibraltar, and the Cook Islands.
* Regions & Oceans that aren't countries, such as the European Union and the Pacific Ocean.
* Genuine cases of missing data, such as Taiwan.


It's important whenever you use __inner joins__ to be __mindful__ that you might be __excluding important data__, especially if you are joining based on columns that aren't linked in the database schema.

There are two less-common join types SQLite does __not support__ that you should be aware of. The first is a __right join__. A right join, as the name indicates, is exactly the opposite of a left join. Where the left join includes all rows in the table before the JOIN clause, the right join includes all rows in the new table in the JOIN clause. We can see a right join in the Venn diagram below:

![img alt](https://s3.amazonaws.com/dq-content/179/venn_right.svg)


The following two queries, one using a left join and one using a right join, produce identical results.

```SQL
SELECT f.name country, c.name city
FROM facts f
LEFT JOIN cities c ON c.facts_id = f.id
LIMIT 5;
```
```SQL
SELECT f.name country, c.name city
FROM cities c
RIGHT JOIN facts f ON f.id = c.facts_id
LIMIT 5;
```

The main reason a __right join would be used__ is in a __complex query__ where you are __joining more than two tables__. In these cases, using a right join is __preferable because__ it can avoid restructuring your whole query to join one table. __Outside of this__, right joins are used __reasonably rarely__, so for simple joins it's better to use a left join than a right as it will be easier for your query to be read and understood by others.

The other join type not supported by SQLite is a __full outer join__. A full outer join will include all rows from the tables on both sides of the join. We can see a full outer join in the Venn diagram below:

![img alt](https://s3.amazonaws.com/dq-content/179/venn_full.svg)

Like right joins, full outer joins are __reasonably uncommon__, and __similar results__ can be achieved using a __union clause__ (which we will teach in the next mission). The standard SQL syntax for an full outer join is:

```
SELECT f.name country, c.name city
FROM cities c
FULL OUTER JOIN facts f ON f.id = c.facts_id
LIMIT 5;
```

When joining __cities__ and __facts__ with a full outer join, the result will be be the same as our left and right joins above, because there are no values in cities.facts_id that don't exist in facts.id.

Let's look at the Venn diagrams of each join type side by side, which should help you compare the differences of each of the four joins we've discussed so far.

![img alt](https://s3.amazonaws.com/dq-content/179/join_venn_diagram.svg)

Next, let's practice using joins to answer some questions about our data.




### 4.2.1.6 Finding the Most Populous Capital Cities

Previously, we've used column names when specifying order for our query results, like so:

```SQL
SELECT name, migration_rate FROM FACTS
ORDER BY migration_rate desc;
```

There is a handy shortcut we can use in our queries which lets us skip the column names, and instead use the order in which the columns appear in the SELECT clause. In this instance, migration_rate is the second column in our SELECT clause so we can just use 2 instead of the column name:

```SQL
SELECT name, migration_rate FROM FACTS
ORDER BY 2 desc;
```

You can use this shortcut in either the ORDER BY or GROUP BY clauses. Be mindful that you want to ensure your queries are still readable, so typing the full column name may be better for more complex queries.

Let's use what we've learned to produce a list of the top 10 capital cities by population. Remember that capital is a boolean column containing 1 or 0, depending on whether a city is a capital or not. We won't specify which join type you should use - you will need to think about what results you require and select an appropriate join type.

#### Instructions

* Write a query that returns the 10 capital cities with the highest population ranked from biggest to smallest population.
* You should include the following columns, in order:
    * capital_city, the name of the city.
    * country, the name of the country the city is from.
    * population, the population of the city.
    
#### Answers
```SQL
SELECT c.name AS capital_city, f.name AS country, c.population
FROM cities c 
INNER JOIN facts f ON c.facts_id = f.id
WHERE c.capital = 1
ORDER BY c.population DESC
LIMIT 10;
```

In [10]:
query = '''
SELECT * 
FROM facts f
LEFT JOIN cities c 
ON f.id = c.facts_id
WHERE c.facts_id is null
'''

print(conn.execute(query).fetchmany(5))

[(92, 'kv', 'Kosovo', 10887, 10887, 0, 1870981, None, None, None, None, None, None, None, None, None), (117, 'mn', 'Monaco', 2, 2, 0, 30535, 0.12, 6.65, 9.24, 3.83, None, None, None, None, None), (123, 'nr', 'Nauru', 21, 21, 0, 9540, 0.55, 24.95, 5.87, 13.63, None, None, None, None, None), (149, 'sm', 'San Marino', 61, 61, 0, 33020, 0.82, 8.63, 8.45, 8.03, None, None, None, None, None), (156, 'sn', 'Singapore', 697, 687, 10, 5674472, 1.89, 8.27, 3.43, 14.05, None, None, None, None, None)]


### 4.2.1.7 Combining joins with Subqueries (Must have a aliase for subqueries when joining tables)

As we learned in the SQL fundamentals course, subqueries can be used to substitute parts of queries, allowing us to find the answers to more complex questions. We can also join to the result of a subquery, just like we could a table.

Here's an example of a using a join and a subquery to produce a table of countries and their capital cities, like we did earlier in the mission.

![img alt](https://s3.amazonaws.com/dq-content/179/explain_subquery.svg)

Reading subqueries can be overwhelming at first, so we'll break down what happens in this example in several steps. The important thing to remember is that the result of any subqueries are always calculated first, so we __read from the inside out__.

* The subquery, in the red box, is calculated first. This simple query selects all columns from cities, filtering rows that are marked as capital cities by having a value for capital of 1.
* The INNER JOIN joins the subquery result, aliased as c, to the facts table based on the ON clause.
* Two columns are selected from the results of the join:
    * f.name, aliased as country.
    * c.name, aliased as capital_city.
* The results are limited to the first 10 rows.

Below is the output of this query:

<table class="dataframe">
<thead>
<tr>
<th>country</th>
<th>capital_city</th>
</tr>
</thead>
<tbody>
<tr>
<td>Aruba</td>
<td>Oranjestad</td>
</tr>
<tr>
<td>Antigua and Barbuda</td>
<td>Saint John'S</td>
</tr>
<tr>
<td>United Arab Emirates</td>
<td>Abu Dhabi</td>
</tr>
<tr>
<td>Afghanistan</td>
<td>Kabul</td>
</tr>
<tr>
<td>Algeria</td>
<td>Algiers</td>
</tr>
<tr>
<td>Azerbaijan</td>
<td>Baku</td>
</tr>
<tr>
<td>Albania</td>
<td>Tirana</td>
</tr>
<tr>
<td>Armenia</td>
<td>Yerevan</td>
</tr>
<tr>
<td>Andorra</td>
<td>Andorra La Vella</td>
</tr>
<tr>
<td>Angola</td>
<td>Luanda</td>
</tr>
</tbody>
</table>

Using this example as a model, we'll write a similar query to find the capital cities with populations of over 10 million.

#### Instructions
* Using a join and a subquery, write a query that returns capital cities with populations of over 10 million ordered from largest to smallest. Include the following columns:
    * capital_city - the name of the city.
    * country - the name of the country the city is the capital of.
    * population - the population of the city.


#### Answers
```SQL
SELECT c.name capital_city, f.name country, c.population population
FROM facts f
INNER JOIN (
            SELECT * FROM cities
            WHERE capital = 1
            AND population > 10000000
           ) c ON c.facts_id = f.id
ORDER BY 3 DESC;
```


### 4.2.1.8 Challenge: Complex Query with Joins and Subqueries

Let's take everything we've learned before and use it to write a more complex query. It's not uncommon to find that 'thinking in SQL' __takes a bit of getting used to__, so don't be discouraged if this challenge takes you a while. It will __get easier with practice__!

When you're writing complex queries with joins and subqueries, it helps to follow this process:

* Think about what data you need in your final output
* Work out which tables you'll need to join, and whether you will need to join to a subquery.
    * If you need to join to a subquery, write the subquery first.
* Then start writing your SELECT clause, followed by the join and any other clauses you will need.
* Don't be afraid to write your query in steps, running it as you go— for instance you can run your subquery as a 'stand alone' query first to make sure it looks like you want before writing the outer query.

We will be writing a query to find the countries where the urban center (city) population is more than half of the country's total population. Our final results will look like this. For expected results, we rounded to six decimal places; however, when running your query, don't worry about rounding the urban_pct column.

<table class="dataframe">
<thead>
<tr>
<th>country</th>
<th>urban_pop</th>
<th>total_pop</th>
<th>urban_pct</th>
</tr>
</thead>
<tbody>
<tr>
<td>Uruguay</td>
<td>1672000</td>
<td>3341893</td>
<td>0.500315</td>
</tr>
<tr>
<td>Congo, Republic of the</td>
<td>2445000</td>
<td>4755097</td>
<td>0.514185</td>
</tr>
<tr>
<td>Brunei</td>
<td>241000</td>
<td>429646</td>
<td>0.560927</td>
</tr>
<tr>
<td>New Caledonia</td>
<td>157000</td>
<td>271615</td>
<td>0.578024</td>
</tr>
<tr>
<td>Virgin Islands</td>
<td>60000</td>
<td>103574</td>
<td>0.579296</td>
</tr>
<tr>
<td>Falkland Islands (Islas Malvinas)</td>
<td>2000</td>
<td>3361</td>
<td>0.595061</td>
</tr>
<tr>
<td>Djibouti</td>
<td>496000</td>
<td>828324</td>
<td>0.598800</td>
</tr>
<tr>
<td>Australia</td>
<td>13789000</td>
<td>22751014</td>
<td>0.606083</td>
</tr>
<tr>
<td>Iceland</td>
<td>206000</td>
<td>331918</td>
<td>0.620635</td>
</tr>
<tr>
<td>Israel</td>
<td>5226000</td>
<td>8049314</td>
<td>0.649248</td>
</tr>
<tr>
<td>United Arab Emirates</td>
<td>3903000</td>
<td>5779760</td>
<td>0.675288</td>
</tr>
<tr>
<td>Puerto Rico</td>
<td>2475000</td>
<td>3598357</td>
<td>0.687814</td>
</tr>
<tr>
<td>Bahamas, The</td>
<td>254000</td>
<td>324597</td>
<td>0.782509</td>
</tr>
<tr>
<td>Kuwait</td>
<td>2406000</td>
<td>2788534</td>
<td>0.862819</td>
</tr>
<tr>
<td>Saint Pierre and Miquelon</td>
<td>5000</td>
<td>5657</td>
<td>0.883861</td>
</tr>
<tr>
<td>Guam</td>
<td>169000</td>
<td>161785</td>
<td>1.044596</td>
</tr>
<tr>
<td>Northern Mariana Islands</td>
<td>56000</td>
<td>52344</td>
<td>1.069846</td>
</tr>
<tr>
<td>American Samoa</td>
<td>64000</td>
<td>54343</td>
<td>1.177705</td>
</tr>
</tbody>
</table>

To help you out, the query you will write will include:

* A join to a subquery.
* A subquery to make a calculation.
* An aggregate function.
* A WHERE clause.
* A CAST expression.

Remember that there are multiple ways to write this query, and the list above is based on the approach we took in our solution.


#### Instructions

* Write a query that generates output as shown above. The query should include:
    * The following columns, in order:
        * country, the name of the country.
        * urban_pop, the sum of the population in major urban areas belonging to that country.
        * total_pop, the total population of the country.
        * urban_pct, the percentage of the popularion within urban areas, calculated by dividing urban_pop by total_pop.
    * Only countries that have an urban_pct greater than 0.5.
    * Rows should be sorted by urban_pct in ascending order.
    
#### Answers
```SQL
SELECT
    f.name country,
    c.urban_pop,
    f.population total_pop,
    (c.urban_pop / CAST(f.population AS FLOAT)) urban_pct
FROM facts f
INNER JOIN (
            SELECT
                facts_id,
                SUM(population) urban_pop
            FROM cities
            GROUP BY 1
           ) c ON c.facts_id = f.id
WHERE urban_pct > .5
ORDER BY 4 ASC;
```

### 4.2.1.9 Next Steps

In this mission we learned:

* The difference between inner and left joins.
* How to choose which join is appropriate for your task.
* Using joins with subqueries, aggregate functions and other SQL techniques.

In the next mission, we're going to keep practicing using joins and learn some advanced joining techniques, including:

* Queries with more than one join.
* Nesting and recursive joins.
* Joining data using __UNION__
* Using __CASE__ to categorize data.

## 4.2.2 Intermediate Joins in SQL

### 4.2.2.1 Working With Larger Databases

In the previous mission we learned how to use joins to combine data from two tables within a database. In reality, most databases have more than 2 tables, so we'll need strategies to be able to write queries to __combine data from 3 or more tables__.

In this mission we'll learn some new techniques to work with the sort of databases that most businesses will use. We'll be working with a modified version of a database called __[Chinook](https://github.com/lerocha/chinook-database)__. The Chinook database contains information about a __fictional digital music shop__ - kind of like a mini-iTunes store.

The Chinook database contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in __eleven tables__. Like with our Factbook database, a __schema diagram__ will help us understand the available columns and the structure of the data. Here's a schema diagram for the Chinook database:

![img alt](https://s3.amazonaws.com/dq-content/189/chinook-schema.svg)

Looking at all those tables can be overwhelming at first, but generally speaking we will only need to think about the specific tables that have the data we require and their connections. Before we look at some specific tables, let's take a moment to understand the different parts of the schema diagram.

* Tables names are shown in bold, with the columns in each table listed below.
* Each table has one or more columns with shading, which indicates those columns are a __primary key__. We'll learn more about primary keys in a later mission, but for now all you need to know is that each row's primary key must be __unique__.
* Relationships between tables are shown using lines between the tables. The lines indicate which columns are related. You may notice that __at least one 'end'__ of the relationship will be a primary key.

As you work through this mission, you may need to refer back to the schema diagram. This is normal, so don't be bothered when that happens. It can also be helpful to write a simple query to check the column names and some example data from any of the tables, for instance:

```SQL
SELECT * FROM album LIMIT 3;
```
<table class="dataframe">
<thead>
<tr>
<th>album_id</th>
<th>title</th>
<th>artist_id</th>
</tr>
</thead>
<tbody>
<tr>
<td>1</td>
<td>For Those About To Rock We Salute You</td>
<td>1</td>
</tr>
<tr>
<td>2</td>
<td>Balls to the Wall</td>
<td>2</td>
</tr>
<tr>
<td>3</td>
<td>Restless and Wild</td>
<td>2</td>
</tr>
</tbody>
</table>


### 4.2.2.2 Joining Three Tables

Our first task is to gather some information on a specific purchase. For one single purchase (__invoice_id__) we want to know, for each track purchased:

* The id of the track.
* The name of the track.
* The name of media type of the track.
* The price that the customer paid for the track.
* The quantity of the track that was purchased.

To gather this information, we will need to write a query that __joins 3 tables__: __invoice_line, track, and media_type__. To make this easier, a schema diagram of just these three tables is below:

![img alt](https://s3.amazonaws.com/dq-content/189/schema_il_t_mt.svg)


If you didn't explore these 3 tables in the previous screen, take a moment now to explore them:

* For each of invoice_line, track, and media_type:
    * Write a simple query, using either the console or code box, to view the first few rows of each of these tables.
    * Look at the result of the query to become familiar with the columns and what sort of data we are working with.


Now that you're familiar with the data, we can look at the syntax for joining data from more than 2 tables.

```SQL
SELECT [column_names] 
FROM [table_name_one]
[join_type] JOIN [table_name_two] ON [join_constraint]
[join_type] JOIN [table_name_three] ON [join_constraint];
```

Joining multiple tables is as simple as adding an extra JOIN clause. The SQL engine __interprets joins in order__, so the first join will be executed, and then the second join will be __executed against the result of the first join__. Because of this, we can first build our query in steps:

* with 0 joins.
* with 1 join.
* with 2 joins.

We will use the invoice_line table in our FROM clause, since it contains __3 of the 5 columns we want__ in our final query. You don't have to follow this pattern but if you're not sure, it's __a good place to start__. Since our tasks involves looking for information about a specific invoice_id, let's choose an invoice_id value of 3. Selecting all lines from invoice_line with an invoice_id is straightforward:

```SQL
SELECT * FROM invoice_line
WHERE invoice_id = 3;
```

<table class="dataframe">
<thead>
<tr>
<th>invoice_line_id</th>
<th>invoice_id</th>
<th>track_id</th>
<th>unit_price</th>
<th>quantity</th>
</tr>
</thead>
<tbody>
<tr>
<td>27</td>
<td>3</td>
<td>2516</td>
<td>0.99</td>
<td>1</td>
</tr>
<tr>
<td>28</td>
<td>3</td>
<td>2646</td>
<td>0.99</td>
<td>1</td>
</tr>
</tbody>
</table>

Now we can use an __inner join__ to add the data from the __track table__.

```SQL
SELECT * FROM invoice_line il
INNER JOIN track t ON t.track_id = il.track_id
WHERE invoice_id = 3;
```

<table class="dataframe">
<thead>
<tr>
<th>invoice_line_id</th>
<th>invoice_id</th>
<th>track_id</th>
<th>unit_price</th>
<th>quantity</th>
<th>track_id</th>
<th>name</th>
<th>album_id</th>
<th>media_type_id</th>
<th>genre_id</th>
<th>composer</th>
<th>milliseconds</th>
<th>bytes</th>
<th>unit_price</th>
</tr>
</thead>
<tbody>
<tr>
<td>27</td>
<td>3</td>
<td>2516</td>
<td>0.99</td>
<td>1</td>
<td>2516</td>
<td>Black Hole Sun</td>
<td>203</td>
<td>1</td>
<td>1</td>
<td>Soundgarden</td>
<td>320365</td>
<td>10425229</td>
<td>0.99</td>
</tr>
<tr>
<td>28</td>
<td>3</td>
<td>2646</td>
<td>0.99</td>
<td>1</td>
<td>2646</td>
<td>I Looked At You</td>
<td>214</td>
<td>1</td>
<td>1</td>
<td>Robby Krieger, Ray Manzarek, John Densmore, Ji...</td>
<td>142080</td>
<td>4663988</td>
<td>0.99</td>
</tr>
</tbody>
</table>

![img alt](https://s3.amazonaws.com/dq-content/189/multiple_joins_step1.svg)

Next, we add our __second join__ to add the data from the __media_type table__.

```SQL
SELECT * FROM invoice_line il
INNER JOIN track t ON t.track_id = il.track_id
INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
WHERE invoice_id = 3;
```

<table class="dataframe">
<thead>
<tr>
<th>invoice_line_id</th>
<th>invoice_id</th>
<th>track_id</th>
<th>unit_price</th>
<th>quantity</th>
<th>track_id</th>
<th>name</th>
<th>album_id</th>
<th>media_type_id</th>
<th>genre_id</th>
<th>composer</th>
<th>milliseconds</th>
<th>bytes</th>
<th>unit_price</th>
<th>media_type_id</th>
<th>name</th>
</tr>
</thead>
<tbody>
<tr>
<td>27</td>
<td>3</td>
<td>2516</td>
<td>0.99</td>
<td>1</td>
<td>2516</td>
<td>Black Hole Sun</td>
<td>203</td>
<td>1</td>
<td>1</td>
<td>Soundgarden</td>
<td>320365</td>
<td>10425229</td>
<td>0.99</td>
<td>1</td>
<td>MPEG audio file</td>
</tr>
<tr>
<td>28</td>
<td>3</td>
<td>2646</td>
<td>0.99</td>
<td>1</td>
<td>2646</td>
<td>I Looked At You</td>
<td>214</td>
<td>1</td>
<td>1</td>
<td>Robby Krieger, Ray Manzarek, John Densmore, Ji...</td>
<td>142080</td>
<td>4663988</td>
<td>0.99</td>
<td>1</td>
<td>MPEG audio file</td>
</tr>
</tbody>
</table>

![img alt](https://s3.amazonaws.com/dq-content/189/multiple_joins_step2.svg)

The last step is to alter the SELECT clause to include only the columns we require - let's do that now with a different order.


#### Instructions
* Write a query that gathers data about the invoice with an invoice_id of 4. Include the following columns in order:
    * The id of the track, track_id.
    * The name of the track, track_name.
    * The name of media type of the track, track_type.
    * The price that the customer paid for the track, unit_price.
    * The quantity of the track that was purchased, quantity.


#### Answers
```SQL
SELECT
    il.track_id,
    t.name track_name,
    mt.name track_type,
    il.unit_price,
    il.quantity
FROM invoice_line il
INNER JOIN track t ON t.track_id = il.track_id
INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
WHERE il.invoice_id = 4;
```



### 4.2.2.3 Joining More Than Three Tables (Finding a Path to link the specific columns we care about)

Let's extend the query we wrote in the previous screen by adding the artist for each track. If you examine the schema, you'll notice that the data for the artist's name is __not directly connected__ to the __track table__.

![img alt](https://s3.amazonaws.com/dq-content/189/schema_t_al_ar.svg)

In this case, we will need to join two new tables to our existing query:

* __artist__, which contains the artist name data that we need
* __album__, which has a column common to each of the artist and track tables which allows us to join those two tables.

Our select clause won't actually use any of the columns from the album table. This is quite common when writing more complex queries to need to join a specific table because it will let you join to another table.

The query from the previous screen is included here:

```SQL
SELECT
    il.track_id,
    t.name track_name,
    mt.name track_type,
    il.unit_price,
    il.quantity
FROM invoice_line il
INNER JOIN track t ON t.track_id = il.track_id
INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
WHERE il.invoice_id = 4;
```

#### Instructions

* Add a column containing the artists name to the query from the previous screen.
    * The column should be called artist_name
    * The column should be placed between track_name and track_type


#### Answers (Join in Order, the following join is based on the previous results)

```SQL
SELECT
    il.track_id,
    t.name track_name,
    ar.name artist_name,
    mt.name track_type,
    il.unit_price,
    il.quantity
FROM invoice_line il
INNER JOIN track t ON t.track_id = il.track_id
INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
INNER JOIN album al ON al.album_id = t.album_id
INNER JOIN artist ar on ar.artist_id = al.artist_id
WHERE il.invoice_id = 4;
```


### 4.2.2.4 Combining Multiple Joins with Subqueries

Because the __invoice_line__ table contains each individual song from each customer purchase, it contains information about which songs are purchased the most. We can use the table to find out which artists are purchased the most. Specifically, what we want to produce is a query that lists the top 10 artists, calculated by the number of times a track by that artist has been purchased.

We'll need to use a __GROUP BY__ clause to get the number of tracks purchased from each artist, but before we do we'll have to join a few tables. As we start thinking about how we will write this query, we can look at the database schema from the first screen of this mission and think about what tables we will need to include.

Writing our query would be a lot easier if we had one table that contained both the __track.track_id and the artist.name__ columns. We can write a __subquery__ that creates this table for us, and then use that subquery to calculate our answer. This means our process will be:

* Write a subquery that produces a table with track.track_id and artist.name,
* Join that subquery to the invoice_line table,
* Use a GROUP BY statement to calculate the number of times each artist has had a track purchased, and find the top 10.

We can write our subquery by joining album to track and then artist to album, just like we did in the previous screen. We'll add an ORDER BY and LIMIT to our query so we're only looking at manageable sample of the data, but we'll remove it when we move to the next step.

```SQL
SELECT
    t.track_id,
    ar.name artist_name
FROM track t
INNER JOIN album al ON al.album_id = t.album_id
INNER JOIN artist ar ON ar.artist_id = al.artist_id
ORDER BY 1 LIMIT 5;
```

<table class="dataframe">
<thead>
<tr>
<th>track_id</th>
<th>artist_name</th>
</tr>
</thead>
<tbody>
<tr>
<td>1</td>
<td>AC/DC</td>
</tr>
<tr>
<td>2</td>
<td>Accept</td>
</tr>
<tr>
<td>3</td>
<td>Accept</td>
</tr>
<tr>
<td>4</td>
<td>Accept</td>
</tr>
<tr>
<td>5</td>
<td>Accept</td>
</tr>
</tbody>
</table>

Next, we need to __join this subquery__ to our __invoice_line table__. We'll give our subquery an __alias ta__ for 'track artist' to make it easier to refer to. Again, we'll add an ORDER BY and LIMIT statement to this step so our output is more manageable.

```SQL
SELECT
    il.invoice_line_id,
    il.track_id,
    ta.artist_name
FROM invoice_line il
INNER JOIN (
            SELECT
                t.track_id,
                ar.name artist_name
            FROM track t
            INNER JOIN album al ON al.album_id = t.album_id
            INNER JOIN artist ar ON ar.artist_id = al.artist_id
           ) ta
           ON ta.track_id = il.track_id
ORDER BY 1 LIMIT 5;
```

<table class="dataframe">
<thead>
<tr>
<th>il.invoice_line_id</th>
<th>il.track_id</th>
<th>ta.artist_name</th>
</tr>
</thead>
<tbody>
<tr>
<td>1</td>
<td>1158</td>
<td>Guns N' Roses</td>
</tr>
<tr>
<td>2</td>
<td>1159</td>
<td>Guns N' Roses</td>
</tr>
<tr>
<td>3</td>
<td>1160</td>
<td>Guns N' Roses</td>
</tr>
<tr>
<td>4</td>
<td>1161</td>
<td>Guns N' Roses</td>
</tr>
<tr>
<td>5</td>
<td>1162</td>
<td>Guns N' Roses</td>
</tr>
</tbody>
</table>

At first it might look like we've done something wrong, because the artist for all rows is Guns N' Roses, but that's because the very first order in our table is a customer who purchased an entire Guns N' Roses album! All that remains now is for us to add our __GROUP BY__ clause, remove the extra columns and use ORDER BY and LIMIT clauses to select the 10 most popular artists.

```SQL
SELECT
    ta.artist_name artist,
    COUNT(*) tracks_purchased
FROM invoice_line il
INNER JOIN (
            SELECT
                t.track_id,
                ar.name artist_name
            FROM track t
            INNER JOIN album al ON al.album_id = t.album_id
            INNER JOIN artist ar ON ar.artist_id = al.artist_id
           ) ta
           ON ta.track_id = il.track_id
GROUP BY 1
ORDER BY 2 DESC LIMIT 10;
```

<table class="dataframe">
<thead>
<tr>
<th>artist</th>
<th>tracks_purchased</th>
</tr>
</thead>
<tbody>
<tr>
<td>Queen</td>
<td>192</td>
</tr>
<tr>
<td>Jimi Hendrix</td>
<td>187</td>
</tr>
<tr>
<td>Nirvana</td>
<td>130</td>
</tr>
<tr>
<td>Red Hot Chili Peppers</td>
<td>130</td>
</tr>
<tr>
<td>Pearl Jam</td>
<td>129</td>
</tr>
<tr>
<td>AC/DC</td>
<td>124</td>
</tr>
<tr>
<td>Guns N' Roses</td>
<td>124</td>
</tr>
<tr>
<td>Foo Fighters</td>
<td>121</td>
</tr>
<tr>
<td>The Rolling Stones</td>
<td>117</td>
</tr>
<tr>
<td>Metallica</td>
<td>106</td>
</tr>
</tbody>
</table>

You can use a similar technique to work out from which albums the most songs have been purchased.


#### Instructions
* Write a query that returns the top 5 albums, as calculated by the number of times a track from that album has been purchased. Your query should return the following columns, in order:
    * album, the title of the album
    * artist, the artist who produced the album
    * tracks_purchased the total number of tracks purchased from that album
* Your query should list the albums from most tracks purchased to least tracks purchased.


#### Answers
```SQL
SELECT 
    ta.album_name album, 
    ta.artist_name artist, 
    COUNT(*) tracks_purchased
FROM invoice_line il
INNER JOIN (
            SELECT t.track_id, ar.name artist_name, al.title album_name
            FROM track t 
            INNER JOIN album al ON al.album_id=t.album_id
            INNER JOIN artist ar ON ar.artist_id=al.artist_id
            ) AS ta
            ON il.track_id = ta.track_id
GROUP BY album, artist
ORDER BY tracks_purchased DESC
LIMIT 5;
           
```

```SQL
Check Tables in a database:

SELECT * FROM chinook.sqlite_master WHERE type='table';
```

In [13]:
conn = sqlite3.connect('chinook.db')

query = '''
SELECT 
    ta.album_name album, 
    ta.artist_name artist, 
    COUNT(*) tracks_purchased
FROM invoice_line il
INNER JOIN (
            SELECT t.track_id, ar.name artist_name, al.title album_name
            FROM track t 
            INNER JOIN album al ON al.album_id=t.album_id
            INNER JOIN artist ar ON ar.artist_id=al.artist_id
            ) AS ta
            ON il.track_id = ta.track_id
GROUP BY album, artist
ORDER BY tracks_purchased DESC
LIMIT 5;
'''

#query_table = "SELECT * FROM chinook.sqlite_master WHERE type='table';"
#print(conn.execute(query_table).fetchall())
print(conn.execute(query).fetchall())


[('Are You Experienced?', 'Jimi Hendrix', 187), ('Faceless', 'Godsmack', 96), ('Mezmerize', 'System Of A Down', 93), ('Get Born', 'JET', 90), ('The Doors', 'The Doors', 83)]


### 4.2.2.5 Recursive Joins

In some cases, there can be a __relation between two columns within the same table__. We can see that in our employee table, where there is a reports_to column that has a relation to the employee_id column within the same table. You might like to explore the employee table using some simple queries in the console.

The reports_to column identifies each employee's supervisor. If we wanted to create a report of each employee and their supervisor's name, we would need some way of __joining a table to itself__. Doing this is called a __recursive join__.

Technically, a recursive join will use one of the other standard joins— usually an __INNER JOIN or LEFT JOIN__ since these are the most commonly used joins— and __aliases to distinguish__ between the table on each side of the join. Here's a simple example of a recursive join in action:

```SQL
SELECT
    e1.employee_id,
    e2.employee_id supervisor_id
FROM employee e1
INNER JOIN employee e2 on e1.reports_to = e2.employee_id
LIMIT 4;
```

This query produces the following table:

<table class="dataframe">
<thead>
<tr>
<th>employee_id</th>
<th>supervisor_id</th>
</tr>
</thead>
<tbody>
<tr>
<td>2</td>
<td>1</td>
</tr>
<tr>
<td>6</td>
<td>1</td>
</tr>
<tr>
<td>3</td>
<td>2</td>
</tr>
<tr>
<td>4</td>
<td>2</td>
</tr>
</tbody>
</table>

To make our report more meaningful, we'll need to add some extra columns. One thing that would be nice is being able to combine the first_name and last_name columns into a single column. We can do that using the concatenate operator: ||.

You can explore how the __concatenate operator__ works using the console:

```SQL
>>> SELECT ("this" || "is" || "my" || "string");
​
    thisismystring
```

Here's a few different examples of using __concatenation__, this time with columns from the album table:

```SQL
SELECT
    album_id,
    artist_id,
    "album id is" || album_id col_1,
    "artist id is" || artist_id col2,
    album_id || artist_id col3
FROM album LIMIT 3;
```

<table class="dataframe">
<thead>
<tr>
<th>album_id</th>
<th>artist_id</th>
<th>col_1</th>
<th>col2</th>
<th>col3</th>
</tr>
</thead>
<tbody>
<tr>
<td>1</td>
<td>1</td>
<td>album id is1</td>
<td>artist id is1</td>
<td>11</td>
</tr>
<tr>
<td>4</td>
<td>1</td>
<td>album id is4</td>
<td>artist id is1</td>
<td>41</td>
</tr>
<tr>
<td>2</td>
<td>2</td>
<td>album id is2</td>
<td>artist id is2</td>
<td>22</td>
</tr>
</tbody>
</table>

You'll notice that the SQL engine will concatenate multiple columns, columns with a string, and that the SQL engine handles converting different types where needed.

Let's use what we've learned about __recursive joins__ and the __concatenation operator__ to create a report about employees and their supervisors.

#### Instructions
* Write a query that returns information about each employee and their supervisor. Your query should return the following columns, in order:
    * employee_name - containing the first_name and last_name columns separated by a space, eg Luke Skywalker
    * employee_title - the title of that employee
    * supervisor_name - the first and last name of the person the employee reports to, in the same format as employee_name
    * supervisor_title - the title of the person the employee reports to
* The report should include employees even if they do not report to another employee.
* The report should be sorted alphabetically by the employee_name column.

#### Answers
```SQL
SELECT 
    e1.first_name || " " || e1.last_name employee_name,
    e1.title employee_title,
    e2.first_name || " " || e2.last_name supervisor_name,
    e2.title supervisor_title
FROM employee e1 
    LEFT JOIN employee e2 
    ON e2.employee_id = e1.reports_to
ORDER BY employee_name
```


In [32]:
### use dir(object) to see if __iter__(), __next()__ exist. If so, the object is a iterable and iterator respectively.
query_emp = 'SELECT * FROM employee LIMIT 10;'
query = '''
SELECT 
    e1.first_name || " " || e1.last_name employee_name,
    e1.title employee_title,
    e2.first_name || " " || e2.last_name supervisor_name,
    e2.title supervisor_title
FROM employee e1 
    LEFT JOIN employee e2 
    ON e2.employee_id = e1.reports_to
ORDER BY employee_name
'''
table_emp = conn.execute(query_emp)

for i in range(5):
    print(table_emp.fetchone())
    
print('\n')
table_rec = conn.execute(query_emp)
results = table_rec.fetchmany(10)
for row in results:
    print(row)
    


(1, 'Adams', 'Andrew', 'General Manager', None, '1962-02-18 00:00:00', '2016-08-14 00:00:00', '11120 Jasper Ave NW', 'Edmonton', 'AB', 'Canada', 'T5K 2N1', '+1 (780) 428-9482', '+1 (780) 428-3457', 'andrew@chinookcorp.com')
(2, 'Edwards', 'Nancy', 'Sales Manager', 1, '1958-12-08 00:00:00', '2016-05-01 00:00:00', '825 8 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 2T3', '+1 (403) 262-3443', '+1 (403) 262-3322', 'nancy@chinookcorp.com')
(3, 'Peacock', 'Jane', 'Sales Support Agent', 2, '1973-08-29 00:00:00', '2017-04-01 00:00:00', '1111 6 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 5M5', '+1 (403) 262-3443', '+1 (403) 262-6712', 'jane@chinookcorp.com')
(4, 'Park', 'Margaret', 'Sales Support Agent', 2, '1947-09-19 00:00:00', '2017-05-03 00:00:00', '683 10 Street SW', 'Calgary', 'AB', 'Canada', 'T2P 5G3', '+1 (403) 263-4423', '+1 (403) 263-4289', 'margaret@chinookcorp.com')
(5, 'Johnson', 'Steve', 'Sales Support Agent', 2, '1965-03-03 00:00:00', '2017-10-17 00:00:00', '7727B 41 Ave', 'Calgary', 'A

### 4.2.2.6 Pattern Matching Using Like

Let's say you're working as a sales support agent for Chinook, and you get back from lunch and see that one of your colleagues has left a phone message on your desk: "Call Jen". Writing a query to get this information out of the database might seem simple at first:

```SQL
SELECT
    first_name,
    last_name,
    phone
FROM customer
WHERE first_name = "Jen";
```

<table class="dataframe">
<thead>
<tr>
<th>first_name</th>
<th>last_name</th>
<th>phone</th>
</tr>
</thead>
<tbody>
</tbody>
</table>

No rows were returned by your query! Suddenly you realize— the name in the database might __not be exactly "Jen"__, it might be Jenny, Jennifer, Jenna, or something else. Rather than having to keep trying different combinations, we can use the __LIKE operator__ to find __pattern matches__. The syntax for LIKE is as follows:

```SQL
WHERE [column_name] LIKE "[pattern]"
```

Your pattern should be the __substring__ you want to match for, and one or more __% characters__:

* %Jen - will match Jen at the end of a string, eg Sarah-Jen
* Jen% - will match Jen at the start of a string, eg Jenny
* %Jen% - will match Jen anywhere within the string, eg Kris Jenner

We'll use the last pattern to give ourselves the best chance of a match:
```python
SELECT
    first_name,
    last_name,
    phone
FROM customer
WHERE first_name LIKE "%Jen%";
```

<table class="dataframe">
<thead>
<tr>
<th>first_name</th>
<th>last_name</th>
<th>phone</th>
</tr>
</thead>
<tbody>
<tr>
<td>Jennifer</td>
<td>Peterson</td>
<td>+1 (604) 688-2255</td>
</tr>
</tbody>
</table>

Perfect - we have found one match and have Jennifer's number so we can return her call.

Keep in mind that in SQLite __LIKE__ is __case insensitive__, so LIKE "%jen%" will match Jen and JEN and JeN. Other flavors of SQL may be case sensitive, so you may need to use the __[LOWER()](https://stackoverflow.com/questions/18853452/sql-select-like-insensitive-casing)__ function to get a case insensitive match.

Let's use __LIKE__ to find a different customer from our database.

#### Instructions
* You have just returned from lunch to see another phone message on your desk: "Call Belle". Write a query that finds the contact details a customer with a first_name containing Belle from the database.
* Your query should include the following columns, in order:
    * first_name
    * last_name
    * phone

#### Answers
```SQL
SELECT
    first_name,
    last_name,
    phone
FROM customer
where first_name LIKE "%belle%";
```


### 4.2.2.7 Generating Columns With The Case Statement (Create a NEW column)

Sometimes, you'll want to add some sort of __categorization__ to your query. Let's say we wanted to generate a report summarizing each customer purchases. It might be nice to __create a column__ that puts each customer into a category:

* big spender - If the customer's total purchases are greater than \$100.

* small spender - If the customer's total purchases are less than \$40.

* regular - If the customer's total purchases are between \$40 and \$100 (inclusive).

To achieve this, we'll need to use the __case statement__. The case statement acts like a series of __if/then options__ for a new column. The syntax for CASE is:

```SQL
CASE
    WHEN [comparison_1] THEN [value_1]
    WHEN [comparison_2] THEN [value_2]
    ELSE [value_3]
    END
    AS [new_column_name]
```

There can be 1 or more __WHEN__ lines, and the __ELSE__ line is optional— without it, rows that __don't match__ any WHEN will be assigned a __null__ value. Let's start by looking at a simple example. The media_type table contains the name of five different media types:

```SQL
SELECT * FROM media_type;
```

<table class="dataframe">
<thead>
<tr>
<th>media_type_id</th>
<th>name</th>
</tr>
</thead>
<tbody>
<tr>
<td>1</td>
<td>MPEG audio file</td>
</tr>
<tr>
<td>2</td>
<td>Protected AAC audio file</td>
</tr>
<tr>
<td>3</td>
<td>Protected MPEG-4 video file</td>
</tr>
<tr>
<td>4</td>
<td>Purchased AAC audio file</td>
</tr>
<tr>
<td>5</td>
<td>AAC audio file</td>
</tr>
</tbody>
</table>


Let's look at how we can use __CASE__ to __add a new column protected__, which indicates whether each media type is protected.

```SQL
SELECT
    media_type_id,
    name,
    CASE
        WHEN name LIKE '%Protected%' THEN 1
        ELSE 0
        END
        AS protected
FROM media_type;
```

<table class="dataframe">
<thead>
<tr>
<th>media_type_id</th>
<th>name</th>
<th>protected</th>
</tr>
</thead>
<tbody>
<tr>
<td>1</td>
<td>MPEG audio file</td>
<td>0</td>
</tr>
<tr>
<td>2</td>
<td>Protected AAC audio file</td>
<td>1</td>
</tr>
<tr>
<td>3</td>
<td>Protected MPEG-4 video file</td>
<td>1</td>
</tr>
<tr>
<td>4</td>
<td>Purchased AAC audio file</td>
<td>0</td>
</tr>
<tr>
<td>5</td>
<td>AAC audio file</td>
<td>0</td>
</tr>
</tbody>
</table>

In this example, our __CASE statement__ has a single WHEN which looks for a partial match of the string Protected in the name column. Any rows with a match get a value of 1, all other rows get 0.

Let's now put __CASE__ into practice using our initial scenario, where we want to categorize customers by their purchase history.

You'll need to know that you __can't use aliases in the WHEN__ line, so when writing a CASE statement that deals with aggregate functions, you will need to __include the aggregate function__ in each WHEN line.

#### Instructions

* Write a query that summarizes the purchases of each customer. For the purposes of this exercise, we do not have any two customers with the same name.
* Your query should include the following columns, in order:
    * customer_name - containing the first_name and last_name columns separated by a space, eg Luke Skywalker.
    * number_of_purchases, counting the number of purchases made by each customer.
    * total_spent - the sum of customers total purchases made by each customer.
    * customer_category - a column that categorizes the customer based on their total purchases. The column should contain the following values:
        * small spender - If the customer's total purchases are less than \$40.
        * big spender - If the customer's total purchases are greater than \$100.
        * regular - If the customer's total purchases are between \$40 and \$100 (inclusive).
* Order your results by the customer_name column.

#### Answers
```SQL
SELECT
   c.first_name || " " || c.last_name customer_name,
   COUNT(i.invoice_id) number_of_purchases,
   SUM(i.total) total_spent,
   CASE
       WHEN sum(i.total) < 40 THEN 'small spender'
       WHEN sum(i.total) > 100 THEN 'big spender'
       ELSE 'regular'
       END
       AS customer_category
FROM invoice i
INNER JOIN customer c ON i.customer_id = c.customer_id
GROUP BY 1 ORDER BY 1;
```



### 4.2.2.8 Next Steps

In this mission, we learned how to work with databases with more than two tables, including how to:

* write queries that join 3 or more tables
* combine multiple joins with subqueries
* create recursive joins
* use LIKE to incorporate partial matches
* generate conditional columns using CASE

In the next mission, we'll continue to work with the Chinook database while we learn how to:

* write increasingly complex queries
* construct and format queries that are easy to read
* create temporary and permanent views using with and view
* use set operations to combine rows

## 4.2.3 Building and Organizing Complex Queries


### 4.2.3.1 Introduction

In the previous two missions, we've learned a lot about joining data. We've gone from creating basic joins between two tables to making complex joins using multiple tables, subqueries, unusual join types and aggregate functions.

In this mission, we're going to continue to practice constructing complex joins, while also learning how to:

* Build and format your queries for readability
* Creating named subqueries and views
* Combining data using set operations.

Just like the previous mission, we'll be working with the __Chinook database__. So you can easily refer to it, the schema for the Chinook database is provided again below.

![img alt](https://s3.amazonaws.com/dq-content/190/chinook-schema.svg)

### 4.2.3.2 Writing Readable Queries

"Code is read much more often than it is written, so plan accordingly

"Even if you don't intend anybody else to read your code, there's still a very good chance that somebody will have to stare at your code and figure out what it does: That person is probably going to be you, twelve months from now."

—Raymond Chen

Often quoted and paraphrased, this philosophy is especially important when writing SQL, where queries can quickly get visually complex. Taking the time to write your queries to be more easily understood will take a little extra time now, but will save you time when you come back to old queries that you have written, and help your colleagues when you're working in a data team.

One obvious area when it comes to writing queries is the use of __capitalization and whitespace__. Because white space doesn't have any meaning in SQL, it can be used to help convey meaning in a complex query. Let's compare the same query written twice— first without whitespace and capitalization:

```SQL
select ta.artist_name artist, count(*) tracks_sold from invoice_line il
inner join (select t.track_id, ar.name artist_name from track t
inner join album al on al.album_id = t.album_id
inner join artist ar on ar.artist_id = al.artist_id) ta
on ta.track_id = il.track_id group by 1 order by 2 desc limit 10;
```

And now, with whitespace and capitalization:

```SQL
SELECT
    ta.artist_name artist,
    COUNT(*) tracks_sold
FROM invoice_line il
INNER JOIN (
            SELECT
                t.track_id,
                ar.name artist_name
            FROM track t
            INNER JOIN album al ON al.album_id = t.album_id
            INNER JOIN artist ar ON ar.artist_id = al.artist_id
           ) ta
           ON ta.track_id = il.track_id
GROUP BY 1
ORDER BY 2 DESC LIMIT 10;
```

As you can see, a little time put into whitespace and capitalization pays off. A few tips to help make your queries more readable:

* If a select statement has more than one column, put each on a new line, indented from the select statement.
* Always capitalize SQL function names and keywords
* Put each clause of your query on a new line.
* Use indenting to make subqueries appear logically separate.

Another important consideration when writing readable queries is the use of __alias names and shortcuts__. Name aliases should be clear– a common convention is using the first letter of the table name, however if you feel that a query is complex you should consider using more explicit aliases. Similarly, at times lines like GROUP BY 1 can be confusing, and explicitly naming the column will make your query more readable.

If you work in a team, you might consider a SQL style guide— a great guide is available at __[SQL style guide](https://www.sqlstyle.guide/)__, but remember is that readability is more important than consistency. If you have a complex query and you think breaking the style guide will make it more readable, you should do it.

![img alt](https://s3.amazonaws.com/dq-content/190/SQL_style_guide.png)


Throughout the rest of our SQL missions, be mindful of writing queries that are easy to read and understand. While we will continue to check answers based on the results of the queries (rather than taking into account the formatting), practicing this will make your future colleagues (and future self) thank you.

Let's now learn another way to make your queries more readable: __named subqueries__.


### 4.2.3.3 The With Clause

When constructing complex queries, it's useful to create an intermediate table to produce our final results. __You can use subqueries to create these intermediate tables__. <span style="color:red">**Unfortunately**, the way subqueries are written makes it harder to read</span>— the person reading the query needs to find the subquery and read from the inside-out.

One way to alleviate this is to use a __with clause__. With clauses allow you to define one or more named subqueries before the start of the main query. The main query then refers to the subquery by it's alias name, just as if it's a table in the database.

The syntax for the __WITH clause__ is relatively straight-forward.

```SQL
WITH [alias_name] AS ([subquery])
​
SELECT [main_query]
```

Let's look at a simple example, a query designed to gather some info about a the tracks from a single album. First, here's our query written with a __standard subquery and no WITH clause__:

```SQL
SELECT * FROM
    (
     SELECT
         t.name,
         ar.name artist,
         al.title album_name,
         mt.name media_type,
         g.name genre,
         t.milliseconds length_milliseconds
     FROM track t
     INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
     INNER JOIN genre g ON g.genre_id = t.genre_id
     INNER JOIN album al ON al.album_id = t.album_id
     INNER JOIN artist ar ON ar.artist_id = al.artist_id
    )
WHERE album_name = "Jagged Little Pill";
```

By __moving the subquery before the main query__ using a __WITH clause__, the intent of the main query becomes much easier to understand.

```SQL
WITH track_info AS
    (                
     SELECT
         t.name,
         ar.name artist,
         al.title album_name,
         mt.name media_type,
         g.name genre,
         t.milliseconds length_milliseconds
     FROM track t
     INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
     INNER JOIN genre g ON g.genre_id = t.genre_id
     INNER JOIN album al ON al.album_id = t.album_id
     INNER JOIN artist ar ON ar.artist_id = al.artist_id
    )
​
SELECT * FROM track_info
WHERE album_name = "Jagged Little Pill";  ###??? not using track_info.album_name?????
```

While in this example the difference is subtle, using the WITH statement helps a lot when your main query even has some slight complexities. Let's get some practice using __WITH__ in a more complex example.


#### Instructions
* Create a query that shows summary data for every playlist in the Chinook database:
    * Use a WITH clause to create a named subquery with the following info:
        * The unique ID for the playlist.
        * The name of the playlist.
        * The name of each track from the playlist.
        * The length of the each track in seconds.
    * Your final table should have the following columns, in order:
        * playlist_id - the unique ID for the playlist.
        * playlist_name - The name of the playlist.
        * number_of_tracks - A count of the number of tracks in the playlist.
        * length_seconds - The sum of the length of the playlist in seconds.
    * The results should be sorted by playlist_id in ascending order.


#### Answers
```SQL
WITH playlist_info AS
    (
     SELECT
         p.playlist_id,
         p.name playlist_name,
         t.name track_name,
         (t.milliseconds / 1000) length_seconds
     FROM playlist p
     LEFT JOIN playlist_track pt ON pt.playlist_id = p.playlist_id
     LEFT JOIN track t ON t.track_id = pt.track_id
    )

SELECT
    playlist_id,
    playlist_name,
    COUNT(track_name) number_of_tracks,
    SUM(length_seconds) length_seconds
FROM playlist_info
GROUP BY 1, 2
ORDER BY 1;
```

<span style="color:red">**error????**</span>
```SQL
WITH temp_info AS
    (
     SELECT 
        p.playlist_id playlist_id,
        p.name playlist_name,
        COUNT(t.track_id) number_of_tracks,
        SUM(t.milliseconds/1000) length_seconds
     FROM playlist p
        INNER JOIN playlist_track pt
        ON p.playlist_id = pt.playlist_id  ### p.playlist_id and pt.playlist_id SAME NAME!!! This is the reason!!!
        INNER JOIN track t
        ON t.track_id = pt.track_id
     GROUP BY playlist_id, playlist_name  ### ambiguous column name: playlist_id ???????? Why??? alias not working?
    )
SELECT *
FROM temp_info
ORDER BY 1
```


### 4.2.3.4 Creating Views

When we use the __WITH__ clause, we're creating a __temporary named subquery__ that we can use only within that query. But what if we find ourselves using the same WITH with lots of different queries? It would be nice to __permanently define a subquery__ that we can use again and again.

We do this by __creating a view__, which we can then use in all future queries. An easy way to think of this is the WITH clause creates a temporary view. The syntax for creating a view is:

```SQL
CREATE VIEW database.view_name AS
    SELECT * FROM database.table;
```

We'll be specifying the database name using __[database name].[view or table name]__ syntax in instead of just [view or table name]. You'll need to use this in conjunction with any views because we have __[manually attached the database](https://sqlite.org/lang_attach.html)__. If you're working with SQLite on your local machine, or in one of our Jupyter projects, you don't need to specify the database name like in the following example:

```SQL
CREATE VIEW view_name AS
    SELECT * FROM table;
```

Here's an example of how to create a view called customer_2, identical to the existing customer table:

```SQL
CREATE VIEW chinook.customer_2 AS
    SELECT * FROM chinook.customer;
```

If we wanted to __modify this view__, and tried to redefine it, we'd get an __error__:

```SQL
CREATE VIEW chinook.customer_2 AS
    SELECT
        customer_id,
        first_name || last_name name,
        phone,
        email,
        support_rep_id
    FROM chinook.customer;

Error: table trackinfo already exists
```

If we wish to redefine a view, we __first have to delete__, or __drop__ the existing view:

```SQL
DROP VIEW chinook.customer_2;
```

We're going to create two views that give us versions of the customer table where the customers in the view have specific criteria. The first is a view of all customers that live in the USA.

```SQL
CREATE VIEW chinook.customer_usa AS 
     SELECT * FROM chinook.customer
     WHERE country = "USA";
```

We have created this view for you - you can query it using your console or code editor. Once a view is created it acts exactly like a table - you don't need to specify that it's a view when you are querying it, and you can do anything with a view that you could do with a table (keeping in mind that in our interface you'll have to use __[database name].[view_name]__).

Let's create a second view of customers that have purchased more than \$90 from our store.


#### Instructions

* Create a view called customer_gt_90_dollars:
    * The view should contain the columns from customers, in their original order.
    * The view should contain only customers who have purchased more than  \$90 in tracks from the store.
* After the SQL query that creates the view, write a second query to display your newly created view: SELECT * FROM chinook.customer_gt_90_dollars;.
    * Make sure you use a semicolon (;) to indicate the end of each query.


#### Answers
my_code
```SQL
CREATE VIEW chinook.customer_gt_90_dollars AS
    SELECT c.*
    FROM chinook.customer c
    INNER JOIN (
                SELECT c.customer_id
                FROM chinook.customer c
                INNER JOIN chinook.invoice i
                ON c.customer_id = i.customer_id
                GROUP BY c.customer_id
                HAVING SUM(i.total)>90
                ) AS ci
    ON c.customer_id = ci.customer_id;
    
SELECT * 
FROM chinook.customer_gt_90_dollars;
```

answer - <span style="color:red">MUCH BETTER - since table c itself is already a unique combination</span>
```SQL
CREATE VIEW chinook.customer_gt_90_dollars AS 
    SELECT
        c.*
    FROM chinook.invoice i
    INNER JOIN chinook.customer c ON i.customer_id = c.customer_id
    GROUP BY 1
    HAVING SUM(i.total) > 90;
SELECT * FROM chinook.customer_gt_90_dollars;
```

### 4.2.3.5 Combining Rows With Union

We have now created __two views__: __customer_usa__ and __customer_gt_90_dollars__. How can we find customers who are in different permutations of these two views:

* Customers in the USA or have spent more than \$90
* Customers in the USA and have spent more than \$90
* Customers in the USA and have not spent more than \$90

These scenarios require a different type of join as we're wanting to __join rows from not tables__, and not columns. Let's start by looking at just the first scenario, where we want to combine rows that exist in either view.

Where __regular joins__ are used to __join columns__, the __union operator__ is used to __join rows__ from tables and/or views.

![img alt](https://s3.amazonaws.com/dq-content/190/union_vs_join.svg)

The syntax for the union operator is composed of two or more __SELECT__ statements:

```SQL
[select_statement_one]
UNION
[select_statement_two]
```

Rather than using the __ON__ keyword, the statements before and after __UNION__ must have the same number of columns, with compatible types in order. We'll learn more about __types__ in a later mission, but at as an example, FLOAT and INT are __compatible types__, but FLOAT and TEXT are not).


![img alt](https://s3.amazonaws.com/dq-content/190/union_types.svg)

Because we created __customer_usa__ and __customer_gt_90_dollars__ with __identical__ column names, order, and type as customer, we can __safely__ use __UNION__.

To achieve our first scenario (identify customers who are in the USA or have spent more than \$90), the two SELECT statements will be very simple - we can just select all columns and rows from each of the two views.


#### Instructions

* Use __UNION__ to produce table of customers in the USA or have spent more than \$90, using the customer_usa and customer_gt_90_dollars views:
    * The result should contain the columns from customers, in their original order.
    
#### Answers
```SQL

SELECT * from chinook.customer_usa ### Do we need to include the [database] as prefix since it is a VIEW???

UNION

SELECT * from chinook.customer_gt_90_dollars;
```

### 4.2.3.6 Combining Rows Using Intersect and Except

The three scenarios we discussed at the start of the previous screen were:

* Customers who are in the USA __or have__ spent more than \$90
* Customers who are in the USA __and have__ spent more than \$90
* Customers who are in the USA __and have not__ spent more than \$90

We just successfully used __UNION__ for the first, but what about the other two? There are two other operators that will help us with these - __intersect__ and __except__. Combined, these three operators allow us to perform __[set operations](https://en.wikipedia.org/wiki/Set_%28mathematics%29#Basic_operations)__ in SQL. Here's a diagram and explanation of how these compare with union.

![img alt](https://s3.amazonaws.com/dq-content/190/set_operations.svg)

<table>
<thead>
<tr>
<th>Operator</th>
<th>What it Does</th>
<th>Python Equivalent</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>UNION</code></td>
<td>Selects rows that occur in either statement.</td>
<td><code>or</code></td>
</tr>
<tr>
<td><code>INTERSECT</code></td>
<td>Selects rows that occur in both statements.</td>
<td><code>and</code></td>
</tr>
<tr>
<td><code>EXCEPT</code></td>
<td>Selects rows that occur in the first statement, but don't occur in the second statement.</td>
<td><code>and not</code></td>
</tr>
</tbody>
</table>



Both the syntax and the rules about column number and ordering of similar types are the same for INTERSECT and EXCEPT as they are for UNION. This means that identifying customers who are in the USA and have spent more than $90 can be done with the following query:

```SQL
SELECT * from customer_usa
​
INTERSECT
​
SELECT * from customer_gt_90_dollars;
```

Identifying customers who are in the USA and have not spent $90 can be done with the following query:

```SQL
SELECT * from customer_usa
​
EXCEPT
​
SELECT * from customer_gt_90_dollars;
```

The results of __UNION, INTERSECT and EXCEPT__ conform to the <span style="color:red">'everything in SQL is a table'</span> concept we learned in the SQL fundamentals course. The results of these operations can be used in subqueries and joined to other tables for more complex analysis. Let's look at a scenario where we'll need to join the results of a set operation to another table:


#### Instructions

* Write a query that works out how many customers that are in the USA and have purchased more than \$90 are assigned to each sales support agent. For the purposes of this exercise, no two employees have the same name.
* Your result should have the following columns, in order:
    * employee_name - The first_name and last_name of the employee separated by a space, eg Luke Skywalker.
    * customers_usa_gt_90 - The number of customer assigned to that employee that are both from the USA and have have purchased more than \$90 worth of tracks.
* The result should include all employees with the title "Sales Support Agent", but not employees with any other title.
* Order your results by the employee_name column.


#### Answers

my_code
```SQL
WITH temp AS 
    (
    SELECT *
    FROM customer_usa
    INTERSECT
    SELECT *
    FROM customer_gt_90_dollars
    )

SELECT 
    e.first_name || " " || e.last_name employee_name,
    COUNT(temp.customer_id) customers_usa_gt_90
FROM employee e
LEFT JOIN temp
ON e.employee_id = temp.support_rep_id
WHERE e.title = "Sales Support Agent"
GROUP BY employee_name
ORDER BY employee_name
```

DQ code
```SQL
WITH customers_usa_gt_90 AS
    (
     SELECT * FROM customer_usa

     INTERSECT

     SELECT * FROM customer_gt_90_dollars
    )

SELECT
    e.first_name || " " || e.last_name employee_name,
    COUNT(c.customer_id) customers_usa_gt_90  ## Not good! Same name AS WITH clause
FROM employee e
LEFT JOIN customers_usa_gt_90 c ON c.support_rep_id = e.employee_id  ## Not good! 
WHERE e.title = 'Sales Support Agent'
GROUP BY 1 ORDER BY 1;
```



### 4.2.3.7 Mutiple Named Subqueries

When we learned about __WITH__, we said with clauses allow you to define one or more named subqueries, but we didn't show you the syntax for creating more than one named subquery. To do this, you use a __single WITH__ clause and __multiple, comma-separated alias/subquery pairs__:

```SQL
WITH
    [alias_name] AS ([subquery]),
    [alias_name_2] AS ([subquery_2]),
    [alias_name_3] AS ([subquery_3]),
​
SELECT [main_query]
```

While each subquery can be independent, we can __actually use the result of the first subquery in subsequent subqueries__, and so on. This can be a useful way of building readable complex queries.

Let's look at a simple example where we create three named subqueries that build on each other.

```SQL
WITH
    usa AS
        (
        SELECT * FROM customer
        WHERE country = "USA"
        ),
    last_name_g AS
        (
         SELECT * FROM usa
         WHERE last_name LIKE "G%"
        ),
    state_ca AS
        (
        SELECT * FROM last_name_g
        WHERE state = "CA"
        )
​
SELECT
    first_name,
    last_name,
    country,
    state
FROM state_ca
```

<table class="dataframe">
<thead>
<tr>
<th>first_name</th>
<th>last_name</th>
<th>country</th>
<th>state</th>
</tr>
</thead>
<tbody>
<tr>
<td>Tim</td>
<td>Goyer</td>
<td>USA</td>
<td>CA</td>
</tr>
</tbody>
</table>

In reality, we'd usually write this as a single query using multiple __AND operators__ in our WHERE clause, but it helps us demonstrate how multiple subqueries can be defined with a single WITH clause. Let's use a more 'real life' example to gather total sales data on customers from India.

#### Instructions

* Write a query that uses multiple named subqueries in a WITH clause to gather total sales data on customers from India:
    * The first named subquery should return all customers that are from India.
    * The second named subquery should calculate the sum total for every customer.
    * The main query should join the two named subqueries, resulting in the following final columns:
        * customer_name - The first_name and last_name of the customer, separated by a space, eg Luke Skywalker.
        * total_purchases - The total amount spent on purchases by that customer.
    * The results should be sorted by the customer_name column in alphabetical order.


#### Answers
my_code  ## Accumulating Subqueries in WITH Clause
```SQL
WITH 
    customers_india AS
        (
        SELECT *
        FROM customer
        WHERE country='India'
        ),
    sales_per_customer AS 
        (
         SELECT 
            c.first_name || " " || c.last_name customer_name,
            SUM(i.total) total_purchases
         FROM customers_india c
         LEFT JOIN invoice i 
         ON c.customer_id=i.customer_id
         GROUP BY customer_name
        )
SELECT *
FROM sales_per_customer
ORDER BY customer_name
```

DQ_code ## independent subqueries in WITH clause
```SQL
WITH
    customers_india AS
        (
        SELECT * FROM customer
        WHERE country = "India"
        ),
    sales_per_customer AS
        (
         SELECT
             customer_id,
             SUM(total) total
         FROM invoice
         GROUP BY 1
        )

SELECT
    ci.first_name || " " || ci.last_name customer_name,
    spc.total total_purchases
FROM customers_india ci
INNER JOIN sales_per_customer spc ON ci.customer_id = spc.customer_id
ORDER BY 1;
```


### 4.2.3.8 Challenge: Each Country's Best Customer

It's time to bring everything we've learned in the course so far to write a complex query. This query will be a bit harder than anything we've written so far, so don't be discouraged if this challenge takes you a while. Write your query in steps, running it as you go to check on your results— this will make troubleshooting much easier.

__We will be writing a query to find the customer from each country that has spent the most money at our store__. In our database there are no 'ties' for best customer in each country, and we will ignore this case for the exercise.

Our final results will look like this. For expected results, we rounded to two decimal places; however, when running your query, don't worry about rounding the total_purchased column.

<table class="dataframe">
<thead>
<tr>
<th>country</th>
<th>customer_name</th>
<th>total_purchased</th>
</tr>
</thead>
<tbody>
<tr>
<td>Argentina</td>
<td>Diego Gutiérrez</td>
<td>39.60</td>
</tr>
<tr>
<td>Australia</td>
<td>Mark Taylor</td>
<td>81.18</td>
</tr>
<tr>
<td>Austria</td>
<td>Astrid Gruber</td>
<td>69.30</td>
</tr>
<tr>
<td>Belgium</td>
<td>Daan Peeters</td>
<td>60.39</td>
</tr>
<tr>
<td>Brazil</td>
<td>Luís Gonçalves</td>
<td>108.90</td>
</tr>
<tr>
<td>Canada</td>
<td>François Tremblay</td>
<td>99.99</td>
</tr>
<tr>
<td>Chile</td>
<td>Luis Rojas</td>
<td>97.02</td>
</tr>
<tr>
<td>Czech Republic</td>
<td>František Wichterlová</td>
<td>144.54</td>
</tr>
<tr>
<td>Denmark</td>
<td>Kara Nielsen</td>
<td>37.62</td>
</tr>
<tr>
<td>Finland</td>
<td>Terhi Hämäläinen</td>
<td>79.20</td>
</tr>
<tr>
<td>France</td>
<td>Wyatt Girard</td>
<td>99.99</td>
</tr>
<tr>
<td>Germany</td>
<td>Fynn Zimmermann</td>
<td>94.05</td>
</tr>
<tr>
<td>Hungary</td>
<td>Ladislav Kovács</td>
<td>78.21</td>
</tr>
<tr>
<td>India</td>
<td>Manoj Pareek</td>
<td>111.87</td>
</tr>
<tr>
<td>Ireland</td>
<td>Hugh O'Reilly</td>
<td>114.84</td>
</tr>
<tr>
<td>Italy</td>
<td>Lucas Mancini</td>
<td>50.49</td>
</tr>
<tr>
<td>Netherlands</td>
<td>Johannes Van der Berg</td>
<td>65.34</td>
</tr>
<tr>
<td>Norway</td>
<td>Bjørn Hansen</td>
<td>72.27</td>
</tr>
<tr>
<td>Poland</td>
<td>Stanisław Wójcik</td>
<td>76.23</td>
</tr>
<tr>
<td>Portugal</td>
<td>João Fernandes</td>
<td>102.96</td>
</tr>
<tr>
<td>Spain</td>
<td>Enrique Muñoz</td>
<td>98.01</td>
</tr>
<tr>
<td>Sweden</td>
<td>Joakim Johansson</td>
<td>75.24</td>
</tr>
<tr>
<td>USA</td>
<td>Jack Smith</td>
<td>98.01</td>
</tr>
<tr>
<td>United Kingdom</td>
<td>Phil Hughes</td>
<td>98.01</td>
</tr>
</tbody>
</table>

To help you out, the query you will write will include:

* One or more named subqueries defined in a WITH clause
* Aggregate functions like SUM() and MAX()
* Several INNER JOINs
* A subquery to define a column
* GROUP BY and ORDER BY clauses

Remember that there are multiple ways to write this query, and the list above is based on the approach we took in our solution.

#### Instructions

* Create a query to find the customer from each country that has spent the most money at our store. Your query should return the following columns, in order:
    * country - The name of each country that we have a customer from.
    * customer_name - The first_name and last_name of the customer from that country with the most total purchases, separated by a space, eg Luke Skywalker.
    * total_purchased - The total dollar amount that customer has purchased.
* The rows should be order by the country column in alphabetical order.

#### Answers

my_code
```SQL
WITH customer_country AS
    (
    SELECT 
    c.country,
    c.first_name || " " || c.last_name customer_name
    FROM customer c 
    GROUP BY customer_name
    ),
     total_purchases AS
    (
     SELECT 
     c.first_name || " " || c.last_name customer_name,
     SUM(i.total) customer_purchased
     FROM customer c
     INNER JOIN invoice i
     ON c.customer_id=i.customer_id
     GROUP BY customer_name
    )

SELECT
    c.country,
    c.customer_name,
    MAX(t.customer_purchased) total_purchased
FROM customer_country c
INNER JOIN total_purchases t
ON c.customer_name = t.customer_name
GROUP BY c.country
ORDER BY c.country
```

DQ_code
```SQL
WITH
    customer_country_purchases AS
        (
         SELECT
             i.customer_id,
             c.country,
             SUM(i.total) total_purchases
         FROM invoice i
         INNER JOIN customer c ON i.customer_id = c.customer_id
         GROUP BY 1, 2
        ),
    country_max_purchase AS
        (
         SELECT
             country,
             MAX(total_purchases) max_purchase
         FROM customer_country_purchases
         GROUP BY 1
        ),
    country_best_customer AS
        (
         SELECT
            cmp.country,
            cmp.max_purchase,
            (
             SELECT ccp.customer_id
             FROM customer_country_purchases ccp
             WHERE ccp.country = cmp.country AND cmp.max_purchase = ccp.total_purchases
            ) customer_id
         FROM country_max_purchase cmp
        )
SELECT
    cbc.country country,
    c.first_name || " " || c.last_name customer_name,
    cbc.max_purchase total_purchased
FROM customer c
INNER JOIN country_best_customer cbc ON cbc.customer_id = c.customer_id
ORDER BY 1 ASC
```

### 4.2.3.9 Next Steps

Well done on completing the challenge - that was a tough one! In this mission we've:

* continued to practice working with complex joins to answer queries
* learned how to use formatting and whitespace to make our queries more readable
* learned how to use WITH and VIEW to make temporary and permanent views
* learned how to perform set operations in SQL using UNION, INTERSECT, and EXCEPT

In the next mission, we'll put everything we've learned together in a guided project.


## 4.2.5 Table Relations and Normalization (SQLite Shell)

To run shell command in Jupyter Notebook: adding "!" before each shell command.

### 4.2.5.1 The SQLite Shell

So far, we've learned lots of skills to extract data from databases. This mission will teach the other side of the equation - __how to create and modify databases__. Rather than working with a SQL editor, in this mission we'll be working with the __SQLite shell__.

You'll need to have basic knowledge of how to work with the command line - if you haven't already, we recommend completing our command line beginner course.

We'll be continuing to work with the Chinook music store database, which is stored in the database file __chinook.db__. To launch the SQLite shell, you use the __sqlite3__ command followed by the name of the database file as an argument.

```SHELL
/home/dq$ sqlite3 chinook.db
```

When you launch the SQLite shell, you will be shown the SQLite prompt. Let's see what that looks like when we open our __chinook.db__ file:

```SHELL
/home/dq$ sqlite3 chinook.db                  
SQLite version 3.21.0 2017-10-24 18:55:49            
Enter ".help" for usage hints.                       
sqlite>
```

If you write a __query__, you'll get the response displayed in your console. If you press enter, the SQLite prompt will change to __...>__ and you can continue writing your query on multiple lines. As a result, unlike any of the places you've written SQL queries so far, including a __semicolon (;)__ is necessary in the SQLite shell. If you don't use it, the shell will not know if you are finished writing your query. Let's write a query to look at one of our tables from the Chinook database.

```SQL
sqlite> SELECT
   ...>   track_id,
   ...>   name,
   ...>   album_id
   ...> FROM track
   ...> WHERE album_id = 3;
3|Fast As a Shark|3
4|Restless and Wild|3
5|Princess of the Dawn|3
```

The first thing you may notice is that we don't have the column names displayed. SQLite has a number of __dot commands__ which you can use to help you work with databases. When you use a dot command, you don't need to use a semicolon. One that you'll want to use often is __.headers on__, which switches column headers on. Let's see what the output of our query looks like after we turn column headers on:

```SHELL
sqlite> .headers on
sqlite> SELECT
   ...>   track_id,
   ...>   name,
   ...>   album_id
   ...> FROM track
   ...> WHERE album_id = 3;
track_id|name|album_id
3|Fast As a Shark|3
4|Restless and Wild|3
5|Princess of the Dawn|3
```

The next thing is that it's hard to read down the columns, since they don't line up. There's another dot command, __.mode__, that will help us out here. The .mode dot command allows us to select from a few different display modes. We'll use .mode column to allow for easier to read outputs. Here's the output of our query after __switching to column mode__:

```SHELL
sqlite> .mode column
sqlite> SELECT
   ...>   track_id,
   ...>   name,
   ...>   album_id
   ...> FROM track
   ...> WHERE album_id = 3;
track_id  name                  album_id
--------  --------------------  --------
3         Fast As a Shark       3
4         Restless and Wild     3
5         Princess of the Dawn  3
```

There are several other __dot commands__ you'll use often:

* .help - Displays help text showing all dot commands and their function.
* .tables - Displays a list of all tables and views in the current database.
* .shell [command] - Run a command like ls or clear in the system shell.
* .quit - Quits the SQLite shell.

#### Instructions
* Launch the SQLite shell, connecting to the chinook.db database file.
* Use __.tables__ to display the tables in the Chinook database.
* Use dot commands to turn headers on, and set the mode to 'column'.
* Write a query on one of the tables in the chinook.db database.
* Display the help text.
* Run __.shell clear__ to see what it does.
* Quit the SQLite shell.

#### Answers


### 4.2.5.2 Creating Tables

Now that we've learned the basics of the SQLite shell, let's learn how to create a database table. Here's the syntax for __CREATE TABLE__:

```SQL
CREATE TABLE [table_name] (
    [column1_name] [column1_type],
    [column2_name] [column2_type],
    [column3_name] [column3_type],
    [...]
);
```

Each column in SQLite must have a type. While some database systems have as many as 50 distinct data types, SQLite uses only 5 behind the scenes:

* TEXT
* INTEGER
* REAL
* NUMERIC
* BLOB

If you have any experience with other database systems, you might be familiar with other types such as VARCHAR, REAL, and DATETIME. SQLite accepts most common types in a CREATE statement, but __behind the scenes__ will convert them to one of its 5 base types.

The table below shows each of the types, along with examples of data commonly stored in the type, and some 'equivalent' types from other database systems. If you're not familiar with these other types, don't be concerned - we'll cover them in some more detail in a later course.

<table>
<tbody><tr>
<th>Type</th>
<th>Commonly Used For</th>
<th>Equivalent Types</th>
</tr>
<tr>
<td><code>TEXT</code></td>
<td>
        Names<br>
        Email Addresses<br>
        Dates and Times<br>
        Phone Numbers
    </td>
<td>
<code>CHARACTER</code><br>
<code>VARCHAR</code><br>
<code>NCHAR</code><br>
<code>NVARCHAR</code><br>
<code>DATETIME</code>
</td>
</tr>
<tr>
<td><code>INTEGER</code></td>
<td>
        IDs<br>
        Quantities<br>
</td>
<td>
<code>INT</code><br>
<code>SMALLINT</code><br>
<code>BIGINT</code><br>
<code>INT8</code>
</td>
</tr>
<tr>
<td><code>REAL</code></td>
<td>
        Weights<br>
        Averages
    </td>
<td>
<code>DOUBLE</code><br>
<code>FLOAT</code>
</td>
</tr>
<tr>
<td><code>NUMERIC</code></td>
<td>
        Prices<br>
        Statuses
    </td>
<td>
<code>DECIMAL</code><br>
<code>BOOLEAN</code>
</td>
</tr>
<tr>
<td><code>BLOB</code></td>
<td>Binary Data</td>
<td><code>BLOB</code></td>
</tr>
</tbody></table>


Just like with views, if you try to create a table that already exists you will get an error. If you make a mistake when you create a table, you can use the __DROP statement__ to remove the table so you can create it again:

```SQL
DROP TABLE [table_name];
```

You can also use the SQLite dot command __.schema [table_name]__ to view the schema for a table you have just created to check where you might have gone wrong.

To practice, we'll create a new table in a new database file. If you launch the SQLite shell with the argument of a filename that doesn't exist, SQLite will create an empty database with that filename.

Make sure you quit the SQLite shell at the end, as it triggers our answer checking.


#### Instructions
* Launch the SQLite shell and create a new database file, new_database.db.
* Create a new table, user, with the following columns:
    * user_id, with type INTEGER
    * first_name, with type TEXT
    * last_name, with type TEXT
* Use the .schema dot command to view the schema for your newly created table.
* Quit the SQLite shell.


#### Answers

### 4.2.5.3 Primary and Foreign Keys

We have been using schema diagrams to identify the relationships between tables. Below is an excerpt of the schema diagram for the Chinook database which shows the relationship between the __invoice and invoice_line__ tables:

![img alt](https://s3.amazonaws.com/dq-content/192/chinook_pk_fk.svg)

We previously learned that each table has one or more columns shaded in yellow, which indicates they are the __primary key__. A primary key is a unique identifier for each row - you cannot have two rows in a table with the same value for the primary key column(s).

When two tables have a __relation__, there will be a column in one table that is a primary key in another table. For example, in the invoice_line table, the invoice_id column is the primary key from the invoice table. This is known as a __foreign key__. By defining a foreign key, our database engine will __prevent__ us from adding rows where the foreign key value doesn't exist in the other table, which helps to __prevent errors__ in our data (note that by default SQLite doesn't force foreign key constraints, however we have __[changed the default](https://stackoverflow.com/questions/15301643/sqlite3-forgets-to-use-foreign-keys/44857286#44857286)__ for this mission).

Usually, a primary key is specified as part of a create statement. Once the primary key is defined, the database engine will prevent any new rows from being added to the database if they have the same primary key as any existing rows. If we wanted to __re-create__ the table from the previous exercise with a primary key, we would use this syntax:

```SQL
CREATE TABLE user (
    user_id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT
);
```

Let's say we wanted to create a new table __purchase__ which tracks basic information about a purchase made by one of our users. Our create statement might look like this:

```SQL
CREATE TABLE purchase (
    purchase_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    purchase_date TEXT,
    total NUMERIC,
    FOREIGN KEY (user_id) REFERENCES user(user_id)
);
```

By adding a __FOREIGN KEY__ clause, we can define one of our columns as a foreign key and specify the table and column that it references. We're going to use what we've learned about creating tables, primary keys and foreign keys to add new tables to our Chinook database that allow customers to create "wishlists" of tracks they would like to buy.

We'll start by adding a table to store the name of the wishlist and the customer that created the wishlist. The schema is shown below:

![img alt](https://s3.amazonaws.com/dq-content/192/wishlist_1.svg)

#### Instructions
* Launch the SQLite shell, connected to the chinook.db database.
* Create a new table, wishlist, with the following columns:
    * wishlist_id, with type INTEGER.
        * This column should be the primary key.
    * customer_id, with type INTEGER.
        * This column should have a foreign key relationship with the customer_id column from the customer table.
    * name, with type TEXT.
* Quit the SQLite shell.

#### Answers

### 4.2.5.4 Database Normalization

When we created our __wishlist table__, we didn't include a __track_id__ column to store which tracks are in the users wishlist. To understand why, let's take a look at what the table might look like if we stored all the data in a single table.

<table>
<thead>
<tr>
<th>wishlist_id</th>
<th>customer_id</th>
<th>name</th>
<th>track_id</th>
</tr>
</thead>
<tbody>
<tr>
<td>1</td>
<td>34</td>
<td>Joao's awesome wishlist</td>
<td>1158</td>
</tr>
<tr>
<td>1</td>
<td>34</td>
<td>Joao's awesome wishlist</td>
<td>2646</td>
</tr>
<tr>
<td>1</td>
<td>34</td>
<td>Joao's awesome wishlist</td>
<td>1990</td>
</tr>
<tr>
<td>2</td>
<td>18</td>
<td>Amy loves pop</td>
<td>3272</td>
</tr>
<tr>
<td>2</td>
<td>18</td>
<td>Amy loves pop</td>
<td>3470</td>
</tr>
</tbody>
</table>

There are are some __drawbacks__ to storing the data this way:

* __Data Duplication__ - we are storing the name of each wishlist multiple times.
* __Data Modification__ - if we want to change the name of one of the wishlists, we have to modify multiple rows.
* __Data Integrity__ - There is nothing to stop a row being added with the wrong wishlist name, and if that happened we wouldn't know which was the correct name.

The process of optimizing the design of databases to minimize these issues is called __database normalization__. In database normalization theory, there are several different phases of normalization, known as __[normal forms](http://www.bkent.net/Doc/simple5.htm)__. Knowing each normal form is not as important as understanding the goals of normalization, and designing your databases to avoid data duplication and integrity issues. We'll learn more about database normalization in the next mission, however for now let's look at how we can design our wishlist tables with normalization in mind:

![img alt](https://s3.amazonaws.com/dq-content/192/wishlist_2.svg)

In addition to the __wishlist__ table we made in the previous screen, we have added a new __wishlist_track__ table and shown its relationship to the existing __track__ table. The wishlist_track table has two columns that are both yellow to indicate that they're primary keys, since neither column will __uniquely identify__ each row by itself. When two or more columns combine to form a primary key it is called a __compound primary key__. To create a compound primary key, you use the __PRIMARY KEY clause__:

```SQL
CREATE TABLE [table_name] (
    [column_one_name] [column_one_type],
    [column_two_name] [column_two_type],
    [column_three_name] [column_three_type],
    [column_four_name] [column_four_type],
    PRIMARY KEY (column_one_name, column_two_name)
);
```

Both columns in the __wishlist_track__ table also have lines to indicate that they are foreign keys. To create a table with multiple foreign keys, you simply use multiple __FOREIGN KEY clauses__.


#### Instructions
* Launch the SQLite shell, connected to the chinook.db database.
* Create a new table, wishlist_track:
    * The table should have the following columns:
        * wishlist_id, with type INTEGER.
        * track_id, with type INTEGER.
    * A primary key should be specified, composed of both columns from the table.
    * Each of the columns should be designated as a foreign key as indicated in the schema diagram.
* Quit the SQLite shell.

#### Answers
```SQL
# +--------------------------------+
# |                                |
# |     RUN THE COMMANDS BELOW     |
# |                                |
# +--------------------------------+
#
# /home/dq$ sqlite3 chinook.db
#  sqlite3> CREATE TABLE wishlist_track (
#      ...> wishlist_id INTEGER,
#      ...> track_id INTEGER,
#      ...> PRIMARY KEY (wishlist_id, track_id),
#      ...> FOREIGN KEY (wishlist_id) REFERENCES wishlist(wishlist_id),
#      ...> FOREIGN KEY (track_id) REFERENCES track(track_id)
#      ...> );
#  sqlite3> .quit
#
#
# +--------------------------------+
# |                                |
# |     👇 does the same as ☝️     |
# |                                |
# +--------------------------------+

sqlite3 chinook.db <<EOF
CREATE TABLE wishlist_track (
    wishlist_id INTEGER,
    track_id INTEGER,
    PRIMARY KEY (wishlist_id, track_id),
    FOREIGN KEY (wishlist_id) REFERENCES wishlist(wishlist_id),
    FOREIGN KEY (track_id) REFERENCES track(track_id)
);
EOF
```

### 4.2.5.5 Inserting and Deleting Rows

Now that we've created the tables to hold our wishlist data, let's add some rows to those tables. To add rows to a SQL table, we'll use the __INSERT__ statement:

```SQL
INSERT INTO [table_name] (
    [column1_name],
    [column2_name],
    [column3_name]
) VALUES (
    [value1],
    [value2],
    [value3]
);
```

If you are inserting values into every column in a table, you don't need to list the column names:

```SQL
INSERT INTO [table_name] VALUES ([value1], [value2], [value3]);
```

Additionally, you can insert multiple rows in a single statement:

```SQL
INSERT INTO [table_name]
VALUES
    ([value1], [value2], [value3]),
    ([value4], [value5], [value6]),
    [...]
```    

Because of our __foreign key constraints__, we'll need to start by adding rows to the wishlist table, and then add rows to the __wishlist_track__ table. If we don't, our insert statement will fail. At the end, we want our wishlist table to contain this data:

<table>
<thead>
<tr>
<th>wishlist_id</th>
<th>customer_id</th>
<th>name</th>
</tr>
</thead>
<tbody>
<tr>
<td>1</td>
<td>34</td>
<td>Joao's awesome wishlist</td>
</tr>
<tr>
<td>2</td>
<td>18</td>
<td>Amy loves pop</td>
</tr>
</tbody>
</table>

And our wishlist_track table to contain this data:

<table>
<thead>
<tr>
<th>wishlist_id</th>
<th>track_id</th>
</tr>
</thead>
<tbody>
<tr>
<td>1</td>
<td>1158</td>
</tr>
<tr>
<td>1</td>
<td>2646</td>
</tr>
<tr>
<td>1</td>
<td>1990</td>
</tr>
<tr>
<td>2</td>
<td>3272</td>
</tr>
<tr>
<td>2</td>
<td>3470</td>
</tr>
</tbody>
</table>

If you make an error while inserting new rows, you'll need to use the __DELETE statement__ to remove all rows:


```SQL
DELETE FROM [table_name]
```

Or use it with a where statement to remove selected rows:

```SQL
DELETE FROM [table_name]
WHERE [expression];
```


#### Instructions
* Launch the SQLite shell, connected to the chinook.db database.
* Add new rows to the wishlist table, in order, as shown above.
* Add new rows to the wishlist_track table, in order, as shown above.
* Quit the SQLite shell.


#### Answers
```SQL
# +--------------------------------+
# |                                |
# |     RUN THE COMMANDS BELOW     |
# |                                |
# +--------------------------------+
#
# /home/dq$ sqlite3 chinook.db
#  sqlite3> INSERT INTO wishlist
#      ...> VALUES
#      ...>     (1, 34, "Joao's awesome wishlist"),
#      ...>     (2, 18, "Amy loves pop");
#  sqlite3> INSERT INTO wishlist_track
#      ...> VALUES
#      ...>     (1, 1158),
#      ...>     (1, 2646),
#      ...>     (1, 1990),
#      ...>     (2, 3272),
#      ...>     (2, 3470);
#  sqlite3> .quit
#
#
# +--------------------------------+
# |                                |
# |     👇 does the same as ☝️     |
# |                                |
# +--------------------------------+

sqlite3 chinook.db <<EOF
INSERT INTO wishlist
VALUES
    (1, 34, "Joao's awesome wishlist"),
    (2, 18, "Amy loves pop");
INSERT INTO wishlist_track
VALUES
    (1, 1158),
    (1, 2646),
    (1, 1990),
    (2, 3272),
    (2, 3470);
EOF
```


### 4.2.5.6 Adding Columns to a Table

We now have two tables to track our wishlist data, and have seeded some data. But what should we do when a customer wants to remove a track from their wishlist? One approach might be just to DELETE the row from wishlist_track. The downside of this approach is that we don't retain any historical data on which tracks were added to wishlists which reduces our ability to analyze this in the future.

A better approach would be to add a column that has a boolean value to show whether the row is active or not, and just change that value if the user wants to delete a track. We can do a similar thing with the wishlists themselves, so users can delete (or technically, __deactivate__) wishlists they no longer want to use.

We'll need to add a column to each of our tables. We can use the __ALTER statement__ to do this.

```SQL
ALTER TABLE [table_name]
ADD COLUMN [column_name] [column_type];
```

As we learned earlier, SQLite supports only five basic types - the closest thing to a boolean type is __NUMERIC__, where the values 1 and 0 will represent __true and false__ respectively. Let's create active columns for both of our wishlist tables.

#### Instructions
* Launch the SQLite shell, connected to the chinook.db database.
* Create a new column, active, in the wishlist table with type NUMERIC.
* Create a new column, active, in the wishlist_track table with type NUMERIC.
* Quit the SQLite shell.

#### Answers



### 4.2.5.7 Adding Values to Existing Rows

We've added our columns to both wishlist tables, but currently they don't have any data in them. To change values for existing rows, we use the __UPDATE statement__:

```SQL
UPDATE [table_name]
SET [column_name] = [expression]
WHERE [expression]
```

The __WHERE clause__ is optional, and can contain any expression that would be valid in a __SELECT statement__.

There are several variations we can use for our __SET clause__. First we can use a single value:

```SQL
UPDATE customer
SET phone = "+55 (12) 3921-4464"
WHERE customer_id = 1
```

We can use a __subquery that returns a single value__:

```SQL
UPDATE track
SET unit_price = (
                    SELECT AVG(unit_price)
                    FROM track
                 )
```

We can use a __column, or function on an existing column__:

```SQL
UPDATE track
SET unit_price = unit_price * 1.1
```

Lastly, we can __set more than one column at once__:

```SQL
UPDATE wishlist_track
SET
    active = 1,
    purchased = 0;
```

Because our active columns will store a __1 for true and 0 for false__, we'll set the values to 1 for every row.


#### Instructions
* Launch the SQLite shell, connected to the chinook.db database.
* Set all values for the active column in the wishlist table to 1.
* Set all values for the active column in the wishlist_track table to 1.
* Quit the SQLite shell.


#### Answers



### 4.2.5.8 Challenge: Adding Sales Tax Capabilities

Let's put everything we've learned into action with a challenge. Up until now, the Chinook store hasn't had to collect sales tax, but due to some new tax legislation, we will need to charge customers in certain countries taxes on their purchases.

We need to add the following columns to the __invoice table__:

* subtotal
* tax

We also need to update all existing invoices with data for these two new columns. The value for the tax column will need to be 0 for every row, and the value for the subtotal column will need to be the same as the total column for each row.

Here's what the first five rows of those columns will need to look like when you're done:

```SQL
sqlite> SELECT
   ...>    invoice_id,
   ...>    subtotal,
   ...>    tax,
   ...>    total
   ...> FROM invoice
   ...> LIMIT 5;
invoice_id  subtotal    tax         total
----------  ----------  ----------  ----------
1           15.84       0           15.84
2           9.9         0           9.9
3           1.98        0           1.98
4           7.92        0           7.92
```

#### Instructions

* Launch the SQLite shell, connected to the chinook.db database.
* Add two new columns, with values, to the invoice table:
    * tax, with type NUMERIC.
        * The value for all existing rows should be 0.
    * subtotal, with type NUMERIC.
        * The value for each row should be the same as that row's value for total.
* Quit the SQLite shell.

#### Answers
```SQL
# +--------------------------------+
# |                                |
# |     RUN THE COMMANDS BELOW     |
# |                                |
# +--------------------------------+
#
# /home/dq$ sqlite3 chinook.db
#  sqlite3> ALTER TABLE invoice
#      ...> ADD COLUMN tax NUMERIC;
#  sqlite3> ALTER TABLE invoice
#      ...> ADD COLUMN subtotal NUMERIC;
#  sqlite3> UPDATE invoice
#      ...> SET
#      ...>     tax = 0,
#      ...>     subtotal = total;
#  sqlite3> .quit
#
#
# +--------------------------------+
# |                                |
# |     👇 does the same as ☝️     |
# |                                |
# +--------------------------------+

sqlite3 chinook.db <<EOF
ALTER TABLE invoice
ADD COLUMN tax NUMERIC;
ALTER TABLE invoice
ADD COLUMN subtotal NUMERIC;
UPDATE invoice
SET
    tax = 0,
    subtotal = total;
EOF
```


### 4.2.5.9 Next Steps

In this mission, we learned:

* How to work with the SQLite shell.
* How to create new tables and assign primary and foreign keys.
* Basic concepts of database normalization.
* How to insert new rows into tables.
* How to add new columns to existing tables.
* How to update existing data in tables.

In the final guided project of this course, we'll put these new skills into practice by building a normalized database and importing data.